# Bitcoin Tour in Python

This is a re-implementation of Andrej Karpathy's bitcoin implementation in python that can be found in the link below. All codes are his and I just re-did what he did in June 2021.

Reference: http://karpathy.github.io/2021/06/21/blockchain/

## Generate A Crypto Identity
In order to navigate on blockchain, we first need to create a new cryptographic identity, which is just a private, public key pair.
This is essentially digital signature scheme. The one particular scheme used by Bitcoin is called Elliptic Curve Digital Signature Algorithm (ECDSA). The following resource provides an excellent introduction on the subject.
https://andrea.corbellini.name/2015/05/17/elliptic-curve-cryptography-a-gentle-introduction/
The specific curve used in Bitcoin is secp256k1. The following stack exchange explains it really well:
https://bitcoin.stackexchange.com/questions/21907/what-does-the-curve-used-in-bitcoin-secp256k1-look-like

The specific values are specified here: https://en.bitcoin.it/wiki/Secp256k1

Python dataclass object: https://docs.python.org/3/library/dataclasses.html

In [21]:
from IPython.display import Latex

In [54]:
from __future__ import annotations # PEP 563: Postponed Evaluation of Annotations
from dataclasses import dataclass # https://docs.python.org/3/library/dataclasses.html

@dataclass
class Curve:
    """
    Elliptic Curve over the field of integers modulo a price.
    Points on the curve satisfy y^2 = x^3 + a*x + b (mod p).
    """
    
    p: int # the prime modulus of the finite field
    a: int
    b: int
    
# secp256k1 uses a = 0, b = 7, so we're dealing with a cubic curve y^2 = x^3 + 7 (mod p)
bitcoin_curve = Curve(
    p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F,
    a = 0x0000000000000000000000000000000000000000000000000000000000000000, # a = 0
    b = 0x0000000000000000000000000000000000000000000000000000000000000007, # b = 7
)

Now, we have an actual curve. Next we need a Generator point, which is some fixed "starting point" on the curve's cycle. 
Imagine we have a curve. In order to "random walk" around the curve, we have to start from somewhere. 
The generator is a publicly known and agreed upon constant:

In [3]:
@dataclass
class Point:
    """ An integer point (x,y) on a Curve """
    curve: Curve
    x: int
    y: int
    
G = Point(
    bitcoin_curve,
    x = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798,
    y = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8,
)

# We can verify that the generator point is indeed on the curve
print("Generator IS on the curve: ", (G.y**2 - (G.x**3 + 7)) % bitcoin_curve.p == 0)

# We expect some random points will not be on the curve, _MOST_ likely
import random
random.seed(1527)
x = random.randrange(0, bitcoin_curve.p)
y = random.randrange(0, bitcoin_curve.p)
print("Apparently, a totally random point is not: ", (y**2 - x**3 - 7) % bitcoin_curve.p == 0)

Generator IS on the curve:  True
Apparently, a totally random point is not:  False


The elliptic curve parameters over F_P associated with a Koblitz curve secp256k1 are specified by the sextuple T = (p,a,b,G,n,h).
Thus, we need to pre-specify those parameter values for secp256k1 before we can use one.

In [4]:
@dataclass
class Generator:
    """
    A generator over a curve: an initial point and the (pre-computed) order
    """
    G: Point    # a generator point
    n: int      # the order of the generator point, so 0*G = n*G = INF. 
    # Essentially, after n steps, I go back to where I start. In a more formal term, order is the number of all its points.
    
bitcoin_gen = Generator(
    G = G,
    # the order of G is known and can be mathematically derived
    n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141,
)

So far we have defined some data structures and filled them with some publicly known constants related to elliptic curves used in Bitcoin.
Now, we are ready to generate our own private key. The private key is simply a random integer that satisfies 1 <= key < n (recall n is the 
                                                                                                                          order of G):

In [6]:
# secret_key = random.randrange(1, bitcoin_gen.n) # this is how you _would_ do it
secret_key = int.from_bytes(b'Aysajan is cool :P', 'big') # For the purpose of reproducibility, this is how I will do it
assert 1 <= secret_key < bitcoin_gen.n
print(secret_key)

5703626118991814761598925171845964482230864


---
The secret_key generated is our private key. As mentioned, it is just a random integer that satisfies 1 <= key < n.
anyone who knows it can control all of the funds you own on the Bitcoin blockchain.

Next, we are going to generate a public key. The public key is the point on the curve that results from adding the generator point
to itself secret_key times, i.e., we have: public_key = G + G + (secret_key times) + G = secret_key * G. If we look at this form carefully,
we know that the point G is a point on curve and specified by a tuple (x,y), but the secret key is an integer. Therefore, public key is an 
(x,y) tuple public key, again a Point on the Curve. This is where we need to actually define the Addition operator on an elliptic curve.

First we need to understand _greatest common divisor_, or gcd. GCD is the largest positive integer that divides each of the integers. For instance, the gcd for 8 and 12 is 4, that is, gcd(8, 12) = 4. Then here comes __Euclidean Algorithm__, which is an efficient method to compute the greatest common divisor (GCD) of two integers, the largest integer that divides them both without a remainder.

[__Extended Euclidean Algorithm__](https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm) is an extension to the Euclidean algorithm. It computes, in addition to the greatest common divisor (gcd) of integers a and b, also the coefficients of Bezout's identity, which are integers x and y such that

ax + by = gcd(a,b)

---
Next, let's review some of the concepts in modular arithmetic before jumping into the code in the next cell.

Reference: https://www.khanacademy.org/computing/computer-science/cryptography/modarithmetic/a/what-is-modular-arithmetic

When we divide two integers, we will have an expression like this:
    $$\dfrac{A}{B} = Q \text{ remainder}  R $$
In this expression,
A is the dividend
B is the divisor
Q is the quotient
R is the remainder

Sometimes we are only interested in the remainder R and there is specific modular operator (abbreviated as mod) for such purpose. 
Using the same A, B, Q, and R as above, we would have __A mod B = R__

We can say _A modulo B is R_. Where B is referred to as the _modulus_. For example:
$$\dfrac{13}{5}=2 \text{ remainder } 3$$
$$13 \text{ mod } 5 = 3$$

---
One expression that I see a lot is something like this:
$$A\equiv B(\text{mod }C)$$
Let's try to understand what it means. First it reads as __A is congruent to B modulo C__. It is kind cumbersome to read.
We first look at some numbers:

Set 1: {1, 6, 11, 16, 21, 26}
Set 2: {0, 5, 10, 15, 20, 25}

The common thing about set 1 is that every number in the set mod 5 is equal to 1. Similarly, every number in set 2 mod 5 is equal to 0.
In such cases, we say that the numbers in set 1 is in the same equivalence class. The numbers in set 2 is also in the same equivalence class.
Now how do we express mathematically this kind of relationships? Here comes __congruence modulo__:
$$A \equiv B (\text{mod }C)$$

This expression is pronounced as _A is congruent to B modulo C_. Let's examine this expression closer. It has three componets:
1. $\equiv$ is the symbol for congruence, which means the values of A and B are in the same __equivalence class__.
2. (mod C) tells us what operation we applied to A and B.
3. When we have both of these, we call "$\equiv$" __congruence modulo C__.

Let's look at one specific example to better understand this:
$$17 \equiv 7 (mod 5)$$

- 17 mod 5 = 2
- 7 mod 5 = 2

Both 17 and 7 are in the same equivalence class. Be careful that this is different from A mod C: $17 \neq 7 \text{ mod }5$.

Okay, enough of this. Let's look at _modular multiplicative inverse_.

---
In mathematics, a modular multiplicative inverse of an integer $a$ is an integer such that the product $ax$ is congruent to 1
with resepct to modulus $m$. Using the standard notation we used above, it can written as
$$ax \equiv 1(\text{mod }m)$$

Relating this expression to the inv function defined below. When $ax$ and 1 is in the same equivalence class with respect to modulus $p$, it implies
that $ax \text{ mod } p = 1 \text{ mod } p$. We know that $1 \text{ mod }p = 1$, therefore $ax \text{ mod }p = 1$ as well, which can be
re-written as $ax \% p = 1$ where $x$ is modular multiplicative inverse of $a$. This logic is implemented in the function _inv_.

In [17]:
INF = Point(None, None, None) # Special point at "infinity", kind of like a zero.

def extended_euclidean_algorithm(a, b):
    """
    Returns (gcd, x, y) s.t. a * x + b * y == gcd
    This function implements the extended Euclidean 
    algorithm and runs in O(log b) in the worst case,
    taken from Wikipedia.
    """
    
    old_r, r = a, b
    old_s, s = 1, 0
    old_t, t = 0, 1
    while r != 0:
        quotient = old_r // r # When we divide 12 by 2, we get 6. In this example, 12 is called divident, 2 is divisor, and 6 is the quotient
        old_r, r = r, old_r - quotient * r
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t
    return old_r, old_s, old_t

def inv(n, p):
    """Returns modular multiplicate inverse m s.t. (n * m) % p == 1"""
    gcd, x, y = extended_euclidean_algorithm(n, p)
    return x % p

#
def elliptic_curve_addition(self, other: Point) -> Point:
    # handle special case of P + 0 = 0 + P = P
    # Keep in mind here that INF is a special point: INF = Point(None, None, None)
    if self == INF:
        return other
    if other == INF:
        return self
    # handle special case of P + (-P) = 0
    if self.x == other.x and self.y != other.y:
        return INF
    # compute the "slope"
    # The case where P = Q
    if self.x == other.x: # (self.y = other.y is guaranteed too per above check)
        m = (3 * self.x**2 + self.curve.a) * inv(2 * self.y, self.curve.p)
    else:
        m = (self.y - other.y) * inv(self.x - other.x, self.curve.p)
    # compute the new point
    rx = (m**2 - self.x - other.x) % self.curve.p
    ry = (-(m*(rx - self.x) + self.y)) % self.curve.p
    return Point(self.curve, rx, ry)

Point.__add__ = elliptic_curve_addition # Monkey patch addition into the Point class

---
Okay, we need to stop and try to understand this addition function here. I will use one of the posts from Andrea Corbellini to
understand what is going on here.

Source: https://andrea.corbellini.name/2015/05/17/elliptic-curve-cryptography-a-gentle-introduction/

ECC represents __Elliptic Curve Cryptography__. What is an elliptic curve? An elliptic curve is a set of points described by the equation:

$$y^2 = x^3 + ax + b$$
where $4a^3+27b^2\neq 0$. This equation is called _Weierstrass normal form_ for elliptic curves. One feature of elliptic curves is that
they are symmetric about the x-axis.

We also need a point at __infinity__ (also known as ideal point) to be part of the curve. 
This infinity point is denoted by the symbol 0 (zero).

Now, we can explicitly express an elliptic curve formally:
$$\{(x, y)\in \mathbb{R}^2 | y^2 = x^3 + ax + b, 4a^3+27b^2\neq 0\} \cup \{0\}$$

There is a _the group law for elliptic curves_ that defines a group over an elliptic curves. It says:
- the elements of the group are the points of an elliptic curve;
- the __identity element__ is the point at infinity 0;
- the __inverse__ of a point P is the one symmetric about the x-axis;
- __addition__ is given by the following rule: __given three aligned, non-zero points $P$, $Q$, and $R$, their sum is $P+Q+R=0$__.

For the last point, I think we might need a definition:
If three points lie on an elliptic curve E and at the same time also lie on the straight line then their sum is __DEFINED__ to be '0'
the point _at infinity or zero point_.

Then using this definition, how do we find the sum of three points on an elliptic curve? The logic works as follows:
1. First we take two points, let's say $P$ and $Q$ and draw a line that passes throught these two points.
2. This line will intersect a third point on the curve, let's say $R$.
3. We take the inverse of this point, $-R$, we have found the result of $P+Q$.

---
Now let's try to generate some prive-public key pairs. Again recap what we have learned so far. Private key or secret key is just a random
integer. Public key is a point on an elliptic curve. Since public curve is a point, it has x-coordinate value and y-coordinate value.

Public key = secret_key * G = G + G + secret_key number of times + G.

For instance, if sk = 1, then public_key = G; if sk = 5, then public_key = G + G + G + G + G, etc.

In [26]:
# if our secret key was the integer 1, then our public key would just be G:
# Keep in mind that public_key = secret_key * G
sk = 1
pk = G
print(f"secret key: {sk}\n public key: {(pk.x, pk.y)}")
print("Verify the public key is on the curve: ", (pk.y**2 - pk.x**3 - 7) % bitcoin_curve.p == 0)

sk = 2
pk = G + G
print(f"secret key: {sk}\n public key: {(pk.x, pk.y)}")
print("Verify the public key is on the curve: ", (pk.y**2 - pk.x**3 - 7) % bitcoin_curve.p == 0)

sk = 3
pk = G + G + G
print(f"secret key: {sk}\n public key: {(pk.x, pk.y)}")
print("Verify the public key is on the curve: ", (pk.y**2 - pk.x**3 - 7) % bitcoin_curve.p == 0)

secret key: 1
 public key: (55066263022277343669578718895168534326250603453777594175500187360389116729240, 32670510020758816978083085130507043184471273380659243275938904335757337482424)
Verify the public key is on the curve:  True
secret key: 2
 public key: (89565891926547004231252920425935692360644145829622209833684329913297188986597, 12158399299693830322967808612713398636155367887041628176798871954788371653930)
Verify the public key is on the curve:  True
secret key: 3
 public key: (112711660439710606056748659173929673102114977341539408544630613555209775888121, 25583027980570883691656905877401976406448868254816295069919888960541586679410)
Verify the public key is on the curve:  True


---
Thew problem here is that usually secret_key, or private_key is a large integer. Adding G this large number of times would be very slow. 
Fortunately we have this algorithm __double and add__ that can really speed up this process. Andrea's post explains it well. Next, let's implement this algorithm.

In [45]:
def double_and_add(self, k:int) -> Point:
    assert isinstance(k, int) and k >= 0
    result = INF
    append = self
    while k:
        if k & 1:
            result += append
        append += append
        k >>= 1
    return result

# monkey patch double and add into the Point class for convenience
Point.__rmul__ = double_and_add

# "verify" correctness
print(G == 1*G)
print(G + G == 2*G)
print(G + G + G == 3*G)
print(G + G + G + G + G +G + G + G + G + G == 10*G)

True
True
True
True


Maybe, another Python implementation of the algorithm explains the logic better.

Source: https://andrea.corbellini.name/2015/05/17/elliptic-curve-cryptography-a-gentle-introduction/

In [36]:
def bits(n):
    """
    Generates the binary digits of n, starting from the least significant bit.
    
    bits(151) -> 1, 1, 1, 0, 1, 0, 0, 1
    """
    while n:
        yield n & 1 # check out https://stackoverflow.com/questions/231767/what-does-the-yield-keyword-do for "yield"
        n >>= 1 # right shift operator, which is a python bitwise operator
        print(n)
        
def double_and_add_andrea(n, x):
    """
    Returns the result of n * x, computed using
    the double and add algorithm.
    """
    
    result = 0
    addend = x
    
    for bit in bits(n):
        if bit == 1:
            result += addend
        addend *= 2
        
    return result

Now, let's try to derive our actual public key from our secret key defined earlier.

In [47]:
# efficiently derive our actual public key, finally !!
public_key = secret_key * G
print(f"x: {public_key.x}\ny: {public_key.y}")
print("Verify the public key is on the curve: ", (public_key.y**2 - public_key.x**3 - 7) % bitcoin_curve.p == 0)

x: 114156641397826299630828824355167492942493011232318490144835676357671279887432
y: 63096451956329221569248690095904719163504830356429270970001086201206174692979
Verify the public key is on the curve:  True


---
With this private/public key pair we've now generated our crypto identity. Now it is time to derive the associated Bitcoin wallet
address. This wallet address is not public key itself, but it can be deterministically derived from it. In a nutshell, a wallet address
is a hash of public key. Therefore, we need to use somekind of hash function here.

Bitcoin uses SHA-256 and also RIPEMD-160. Therefore, we first implement a SHA-256 hash function. The reference here is the NIST document
[FIPS PUB 180-4](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf).

In [65]:
def gen_sha256_with_variable_scope_protector_to_not_pollute_global_namespace():
    """
    SHA256 implementation.
    
    Follows the FIPS PUB 180-4 description for calculating SHA-256 hash function
    https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
    
    None in their right mind should use this for any serious reason. This was written
    purely for educational purposes.
    """
    
    import math
    from itertools import count, islice
    
    # -------------------------------------------------------------------------------
    # SHA-256 Functions, defined in Section 4 of the FIPS document
    
    # rotate right operations
    def rotr(x, n, size=32):
        return (x >> n) | (x << size - n) & (2**size - 1)
    
    # right shift operation
    def shr(x, n):
        return x >> n
    
    # Sigma0 function
    def sig0(x):
        return rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3)
    
    # Sigma1 function
    def sig1(x):
        return rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10)
    
    # sum0 function
    def capsig0(x):
        return rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22)
    
    # sum1 function
    def capsig1(x):
        return rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25)
    
    # Ch function
    def ch(x, y, z):
        return (x & y) ^ (~x & z)
    
    # Maj function
    def maj(x, y, z):
        return (x & y) ^ (x & z) ^ (y & z)
    
    # bits to integer
    def b2i(b):
        return int.from_bytes(b, 'big')
    
    # integer to bits
    def i2b(i):
        return i.to_bytes(4, 'big')
    
    # ------------------------------------------------------------------------------------
    # SHA-256 Constants
    
    # check if a number is prime number or not
    def is_prime(n):
        return not any(f for f in range(2, int(math.sqrt(n))+1) if n%f == 0)
    
    def first_n_primes(n):
        return islice(filter(is_prime, count(start=2)), n)
    
    def frac_bin(f, n=32):
        """ return the first n bits of fractional part of float f """
        f -= math.floor(f) # get only the fractional part
        f *= 2**n # shift left
        f = int(f) # truncate the rest of the fractional content
        return f
    
    def genK():
        """
        Follows Section 4.2.2 to generate K
        
        The first 32 bits of the fractional parts of the cube roots of the 
        first 64 prime numbers:
        428a2f98 71374491 b5c0fbcf e9b5dba5 3956c25b 59f111f1 923f82a4 ab1c5ed5
        d807aa98 12835b01 243185be 550c7dc3 72be5d74 80deb1fe 9bdc06a7 c19bf174
        e49b69c1 efbe4786 0fc19dc6 240ca1cc 2de92c6f 4a7484aa 5cb0a9dc 76f988da
        983e5152 a831c66d b00327c8 bf597fc7 c6e00bf3 d5a79147 06ca6351 14292967
        27b70a85 2e1b2138 4d2c6dfc 53380d13 650a7354 766a0abb 81c2c92e 92722c85
        a2bfe8a1 a81a664b c24b8b70 c76c51a3 d192e819 d6990624 f40e3585 106aa070
        19a4c116 1e376c08 2748774c 34b0bcb5 391c0cb3 4ed8aa4a 5b9cca4f 682e6ff3
        748f82ee 78a5636f 84c87814 8cc70208 90befffa a4506ceb bef9a3f7 c67178f2
        """
        return [frac_bin(p ** (1/3.0)) for p in first_n_primes(64)]
    
    def genH():
        """
        Follows Section 5.3.3 to generate the initial hash value H^0
        
        The first 32 bits of the fractional parts of the square roots of the 
        first 8 prime numbers.
        
        6a09e667 bb67ae85 3c6ef372 a54ff53a 9b05688c 510e527f 1f83d9ab 5be0cd19
        """
        return [frac_bin(p ** (1/2.0)) for p in first_n_primes(8)]
               
    # -------------------------------------------------------------------------------------
    
    def pad(b):
        """ Follows Section 5.1: Padding the message """
        b = bytearray(b) # convert to a mutable equivalent
        l = len(b) * 8 # note: len returns number of bytes, not bits
        
        # Append but "1" to the end of the message
        b.append(0b10000000) # appending 10000000 in binary (=128 in decimal)
        
        # follow by k zero bits, where k is the smallest non-negative solution to
        # l + 1 + k = 448 mod 512
        # i.e., pad with zeros until we reach 448 (mod 512)
        while (len(b)*8) % 512 != 448:
            b.append(0x00)
            
        # the last 64-bit block is the length l of the original message
        # expressed in binary (big endian)
        b.extend(l.to_bytes(8, 'big'))
        
        return b
   
    def sha256(b: bytes) -> bytes:
        
        """ Follows multiple sections in the FIPS document"""
        # Section 4.2
        K = genK()
        
        # Section 5: Preprocessing
        # Section 5.1: Pad the message
        b = pad(b)
        # Section 5.2: separate the message into blocks of 512 bits (64 bytes)
        blocks = [b[i:i+64]for i in range(0, len(b), 64)]
        
        # for each message block M^1, ..., M^N
        H = genH() # Section 5.3
        
        # Section 6
        for M in blocks: # each block is a 64-entry array of 8-bit bytes
            # 1. Prepare the message schedule, a 64-entry array of 32-bit words
            W = []
            for t in range(64):
                if t <= 15:
                    # The first 16 words are just a copy of the block
                    W.append(bytes(M[t*4:t*4+4]))
                else:
                    term1 = sig1(b2i(W[t-2]))
                    term2 = b2i(W[t-7])
                    term3 = sig0(b2i(W[t-15]))
                    term4 = b2i(W[t-16])
                    total = (term1 + term2 + term3 + term4) % 2**32
                    W.append(i2b(total))
                
            # 2. Initialize the 8 working variables a, b, c, d, e, f, g, h with prev hash value
            a, b, c, d, e, f, g, h = H
            
            # 3. For t = 0 to 63, calculate:
            for t in range(64):
                T1 = (h + capsig1(e) + ch(e, f, g) + K[t] + b2i(W[t])) % 2**32
                T2 = (capsig0(a) + maj(a, b, c)) % 2**32
                h = g
                g = f
                f = e
                e = (d + T1) % 2**32
                d = c
                c = b
                b = a
                a = (T1 + T2) % 2**32
                
            # 4. Compute the i-th intermediate hash valua H^i
            delta = [a, b, c, d, e, f, g, h]
            H = [(i1 + i2) % 2**32 for i1, i2 in zip(H, delta)]
            
        return b''.join(i2b(i) for i in H)
    
    return sha256

sha256 = gen_sha256_with_variable_scope_protector_to_not_pollute_global_namespace()
print("Verify empty hash: ", sha256(b'').hex()) # should be e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
print(sha256(b'here is a random bytes message, cool right?').hex())
print("number of bytes in a sha256 digest: ", len(sha256(b'')))

Verify empty hash:  e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
69b9779edaa573a509999cbae415d3408c30544bad09727a1d64eff353c95b89
number of bytes in a sha256 digest:  32


---
Wow, that is a lot. Let's zoom out and understand the fundamentals of SHA-256. In general the logic is relatively clear. 
SHA256 takes some bytes message (no limit on the size, it can be any size) that is to be hashed, it first pads the message (wait, what does "padding a message" mean? Accoridng to [Wikipedia](https://en.wikipedia.org/wiki/Padding_(cryptography)), "padding is any of a number of
of distinct practices which all include adding data to the beginning, middle, or end of a message prior to encryption."),
then breaks it up into chunks, and passes these chunks into what can be best described as a fancy "bit mixer", defined in Section 3 of FIPS document. This "bit mixer" contains a number of bit shifts and binary operations orchestrated in a way that is hardly intuitive, but that results in a fixed-size, random-looking short digest of any variably-sized original message subject to the scrambling is not invertible and also it is computationaly impossible to construct a different message that hashes to any given digest. But note that it does not mean there are no two original messages that hash to the same digest. As a matter of fact, theoretically there exist two different messages with the same hash value because of a simple fact: The input size of SHA256 > the output size. But it is just computational impossible to actually find that other message that gives us the same hash value.

Bitcoin uses SHA256 everywhere to create hashes. It is also the core element in Bitcoin's Proof of Work (POW), where the goal is to modify
some numbers until the whole thing hashes to a sufficiently low number. This can only be done by brute force search due to the nice properties of SHA256.


Whewwwww. That is a lot. Let's move on from SHA256 to focus on another hash function needed to generate our address in Bitcoin: RIPEMD 160.

---
__RIPEMD 160 hash function__

Description: https://www.esat.kuleuven.be/cosic/publications/article-317.pdf

RIPEMD160 has been superseded by the SHA-256 and SHA-512. I am not sure why it was used in Bitcoin in the first place. 
There is not much historical context about its application in Bitcoin.
For its implementation, Andrea found the
python script on the Internet and shortened and cleaned up. So I will just copy them here without writing line by line and understanding it. Lazy!!

In [66]:
def gen_ripemd160_with_variable_scope_protector_to_not_pollute_global_namespace():

    import sys
    import struct

    # -----------------------------------------------------------------------------
    # public interface

    def ripemd160(b: bytes) -> bytes:
        """ simple wrapper for a simpler API to this hash function, just bytes to bytes """
        ctx = RMDContext()
        RMD160Update(ctx, b, len(b))
        digest = RMD160Final(ctx)
        return digest

    # -----------------------------------------------------------------------------

    class RMDContext:
        def __init__(self):
            self.state = [0x67452301, 0xEFCDAB89, 0x98BADCFE, 0x10325476, 0xC3D2E1F0] # uint32
            self.count = 0 # uint64
            self.buffer = [0]*64 # uchar

    def RMD160Update(ctx, inp, inplen):
        have = int((ctx.count // 8) % 64)
        inplen = int(inplen)
        need = 64 - have
        ctx.count += 8 * inplen
        off = 0
        if inplen >= need:
            if have:
                for i in range(need):
                    ctx.buffer[have+i] = inp[i]
                RMD160Transform(ctx.state, ctx.buffer)
                off = need
                have = 0
            while off + 64 <= inplen:
                RMD160Transform(ctx.state, inp[off:])
                off += 64
        if off < inplen:
            for i in range(inplen - off):
                ctx.buffer[have+i] = inp[off+i]

    def RMD160Final(ctx):
        size = struct.pack("<Q", ctx.count)
        padlen = 64 - ((ctx.count // 8) % 64)
        if padlen < 1 + 8:
            padlen += 64
        RMD160Update(ctx, PADDING, padlen-8)
        RMD160Update(ctx, size, 8)
        return struct.pack("<5L", *ctx.state)

    # -----------------------------------------------------------------------------

    K0 = 0x00000000
    K1 = 0x5A827999
    K2 = 0x6ED9EBA1
    K3 = 0x8F1BBCDC
    K4 = 0xA953FD4E
    KK0 = 0x50A28BE6
    KK1 = 0x5C4DD124
    KK2 = 0x6D703EF3
    KK3 = 0x7A6D76E9
    KK4 = 0x00000000

    PADDING = [0x80] + [0]*63

    def ROL(n, x):
        return ((x << n) & 0xffffffff) | (x >> (32 - n))

    def F0(x, y, z):
        return x ^ y ^ z

    def F1(x, y, z):
        return (x & y) | (((~x) % 0x100000000) & z)

    def F2(x, y, z):
        return (x | ((~y) % 0x100000000)) ^ z

    def F3(x, y, z):
        return (x & z) | (((~z) % 0x100000000) & y)

    def F4(x, y, z):
        return x ^ (y | ((~z) % 0x100000000))

    def R(a, b, c, d, e, Fj, Kj, sj, rj, X):
        a = ROL(sj, (a + Fj(b, c, d) + X[rj] + Kj) % 0x100000000) + e
        c = ROL(10, c)
        return a % 0x100000000, c

    def RMD160Transform(state, block): #uint32 state[5], uchar block[64]

        x = [0]*16
        assert sys.byteorder == 'little', "Only little endian is supported atm for RIPEMD160"
        x = struct.unpack('<16L', bytes(block[0:64]))

        a = state[0]
        b = state[1]
        c = state[2]
        d = state[3]
        e = state[4]

        #/* Round 1 */
        a, c = R(a, b, c, d, e, F0, K0, 11,  0, x)
        e, b = R(e, a, b, c, d, F0, K0, 14,  1, x)
        d, a = R(d, e, a, b, c, F0, K0, 15,  2, x)
        c, e = R(c, d, e, a, b, F0, K0, 12,  3, x)
        b, d = R(b, c, d, e, a, F0, K0,  5,  4, x)
        a, c = R(a, b, c, d, e, F0, K0,  8,  5, x)
        e, b = R(e, a, b, c, d, F0, K0,  7,  6, x)
        d, a = R(d, e, a, b, c, F0, K0,  9,  7, x)
        c, e = R(c, d, e, a, b, F0, K0, 11,  8, x)
        b, d = R(b, c, d, e, a, F0, K0, 13,  9, x)
        a, c = R(a, b, c, d, e, F0, K0, 14, 10, x)
        e, b = R(e, a, b, c, d, F0, K0, 15, 11, x)
        d, a = R(d, e, a, b, c, F0, K0,  6, 12, x)
        c, e = R(c, d, e, a, b, F0, K0,  7, 13, x)
        b, d = R(b, c, d, e, a, F0, K0,  9, 14, x)
        a, c = R(a, b, c, d, e, F0, K0,  8, 15, x) #/* #15 */
        #/* Round 2 */
        e, b = R(e, a, b, c, d, F1, K1,  7,  7, x)
        d, a = R(d, e, a, b, c, F1, K1,  6,  4, x)
        c, e = R(c, d, e, a, b, F1, K1,  8, 13, x)
        b, d = R(b, c, d, e, a, F1, K1, 13,  1, x)
        a, c = R(a, b, c, d, e, F1, K1, 11, 10, x)
        e, b = R(e, a, b, c, d, F1, K1,  9,  6, x)
        d, a = R(d, e, a, b, c, F1, K1,  7, 15, x)
        c, e = R(c, d, e, a, b, F1, K1, 15,  3, x)
        b, d = R(b, c, d, e, a, F1, K1,  7, 12, x)
        a, c = R(a, b, c, d, e, F1, K1, 12,  0, x)
        e, b = R(e, a, b, c, d, F1, K1, 15,  9, x)
        d, a = R(d, e, a, b, c, F1, K1,  9,  5, x)
        c, e = R(c, d, e, a, b, F1, K1, 11,  2, x)
        b, d = R(b, c, d, e, a, F1, K1,  7, 14, x)
        a, c = R(a, b, c, d, e, F1, K1, 13, 11, x)
        e, b = R(e, a, b, c, d, F1, K1, 12,  8, x) #/* #31 */
        #/* Round 3 */
        d, a = R(d, e, a, b, c, F2, K2, 11,  3, x)
        c, e = R(c, d, e, a, b, F2, K2, 13, 10, x)
        b, d = R(b, c, d, e, a, F2, K2,  6, 14, x)
        a, c = R(a, b, c, d, e, F2, K2,  7,  4, x)
        e, b = R(e, a, b, c, d, F2, K2, 14,  9, x)
        d, a = R(d, e, a, b, c, F2, K2,  9, 15, x)
        c, e = R(c, d, e, a, b, F2, K2, 13,  8, x)
        b, d = R(b, c, d, e, a, F2, K2, 15,  1, x)
        a, c = R(a, b, c, d, e, F2, K2, 14,  2, x)
        e, b = R(e, a, b, c, d, F2, K2,  8,  7, x)
        d, a = R(d, e, a, b, c, F2, K2, 13,  0, x)
        c, e = R(c, d, e, a, b, F2, K2,  6,  6, x)
        b, d = R(b, c, d, e, a, F2, K2,  5, 13, x)
        a, c = R(a, b, c, d, e, F2, K2, 12, 11, x)
        e, b = R(e, a, b, c, d, F2, K2,  7,  5, x)
        d, a = R(d, e, a, b, c, F2, K2,  5, 12, x) #/* #47 */
        #/* Round 4 */
        c, e = R(c, d, e, a, b, F3, K3, 11,  1, x)
        b, d = R(b, c, d, e, a, F3, K3, 12,  9, x)
        a, c = R(a, b, c, d, e, F3, K3, 14, 11, x)
        e, b = R(e, a, b, c, d, F3, K3, 15, 10, x)
        d, a = R(d, e, a, b, c, F3, K3, 14,  0, x)
        c, e = R(c, d, e, a, b, F3, K3, 15,  8, x)
        b, d = R(b, c, d, e, a, F3, K3,  9, 12, x)
        a, c = R(a, b, c, d, e, F3, K3,  8,  4, x)
        e, b = R(e, a, b, c, d, F3, K3,  9, 13, x)
        d, a = R(d, e, a, b, c, F3, K3, 14,  3, x)
        c, e = R(c, d, e, a, b, F3, K3,  5,  7, x)
        b, d = R(b, c, d, e, a, F3, K3,  6, 15, x)
        a, c = R(a, b, c, d, e, F3, K3,  8, 14, x)
        e, b = R(e, a, b, c, d, F3, K3,  6,  5, x)
        d, a = R(d, e, a, b, c, F3, K3,  5,  6, x)
        c, e = R(c, d, e, a, b, F3, K3, 12,  2, x) #/* #63 */
        #/* Round 5 */
        b, d = R(b, c, d, e, a, F4, K4,  9,  4, x)
        a, c = R(a, b, c, d, e, F4, K4, 15,  0, x)
        e, b = R(e, a, b, c, d, F4, K4,  5,  5, x)
        d, a = R(d, e, a, b, c, F4, K4, 11,  9, x)
        c, e = R(c, d, e, a, b, F4, K4,  6,  7, x)
        b, d = R(b, c, d, e, a, F4, K4,  8, 12, x)
        a, c = R(a, b, c, d, e, F4, K4, 13,  2, x)
        e, b = R(e, a, b, c, d, F4, K4, 12, 10, x)
        d, a = R(d, e, a, b, c, F4, K4,  5, 14, x)
        c, e = R(c, d, e, a, b, F4, K4, 12,  1, x)
        b, d = R(b, c, d, e, a, F4, K4, 13,  3, x)
        a, c = R(a, b, c, d, e, F4, K4, 14,  8, x)
        e, b = R(e, a, b, c, d, F4, K4, 11, 11, x)
        d, a = R(d, e, a, b, c, F4, K4,  8,  6, x)
        c, e = R(c, d, e, a, b, F4, K4,  5, 15, x)
        b, d = R(b, c, d, e, a, F4, K4,  6, 13, x) #/* #79 */

        aa = a
        bb = b
        cc = c
        dd = d
        ee = e

        a = state[0]
        b = state[1]
        c = state[2]
        d = state[3]
        e = state[4]

        #/* Parallel round 1 */
        a, c = R(a, b, c, d, e, F4, KK0,  8,  5, x)
        e, b = R(e, a, b, c, d, F4, KK0,  9, 14, x)
        d, a = R(d, e, a, b, c, F4, KK0,  9,  7, x)
        c, e = R(c, d, e, a, b, F4, KK0, 11,  0, x)
        b, d = R(b, c, d, e, a, F4, KK0, 13,  9, x)
        a, c = R(a, b, c, d, e, F4, KK0, 15,  2, x)
        e, b = R(e, a, b, c, d, F4, KK0, 15, 11, x)
        d, a = R(d, e, a, b, c, F4, KK0,  5,  4, x)
        c, e = R(c, d, e, a, b, F4, KK0,  7, 13, x)
        b, d = R(b, c, d, e, a, F4, KK0,  7,  6, x)
        a, c = R(a, b, c, d, e, F4, KK0,  8, 15, x)
        e, b = R(e, a, b, c, d, F4, KK0, 11,  8, x)
        d, a = R(d, e, a, b, c, F4, KK0, 14,  1, x)
        c, e = R(c, d, e, a, b, F4, KK0, 14, 10, x)
        b, d = R(b, c, d, e, a, F4, KK0, 12,  3, x)
        a, c = R(a, b, c, d, e, F4, KK0,  6, 12, x) #/* #15 */
        #/* Parallel round 2 */
        e, b = R(e, a, b, c, d, F3, KK1,  9,  6, x)
        d, a = R(d, e, a, b, c, F3, KK1, 13, 11, x)
        c, e = R(c, d, e, a, b, F3, KK1, 15,  3, x)
        b, d = R(b, c, d, e, a, F3, KK1,  7,  7, x)
        a, c = R(a, b, c, d, e, F3, KK1, 12,  0, x)
        e, b = R(e, a, b, c, d, F3, KK1,  8, 13, x)
        d, a = R(d, e, a, b, c, F3, KK1,  9,  5, x)
        c, e = R(c, d, e, a, b, F3, KK1, 11, 10, x)
        b, d = R(b, c, d, e, a, F3, KK1,  7, 14, x)
        a, c = R(a, b, c, d, e, F3, KK1,  7, 15, x)
        e, b = R(e, a, b, c, d, F3, KK1, 12,  8, x)
        d, a = R(d, e, a, b, c, F3, KK1,  7, 12, x)
        c, e = R(c, d, e, a, b, F3, KK1,  6,  4, x)
        b, d = R(b, c, d, e, a, F3, KK1, 15,  9, x)
        a, c = R(a, b, c, d, e, F3, KK1, 13,  1, x)
        e, b = R(e, a, b, c, d, F3, KK1, 11,  2, x) #/* #31 */
        #/* Parallel round 3 */
        d, a = R(d, e, a, b, c, F2, KK2,  9, 15, x)
        c, e = R(c, d, e, a, b, F2, KK2,  7,  5, x)
        b, d = R(b, c, d, e, a, F2, KK2, 15,  1, x)
        a, c = R(a, b, c, d, e, F2, KK2, 11,  3, x)
        e, b = R(e, a, b, c, d, F2, KK2,  8,  7, x)
        d, a = R(d, e, a, b, c, F2, KK2,  6, 14, x)
        c, e = R(c, d, e, a, b, F2, KK2,  6,  6, x)
        b, d = R(b, c, d, e, a, F2, KK2, 14,  9, x)
        a, c = R(a, b, c, d, e, F2, KK2, 12, 11, x)
        e, b = R(e, a, b, c, d, F2, KK2, 13,  8, x)
        d, a = R(d, e, a, b, c, F2, KK2,  5, 12, x)
        c, e = R(c, d, e, a, b, F2, KK2, 14,  2, x)
        b, d = R(b, c, d, e, a, F2, KK2, 13, 10, x)
        a, c = R(a, b, c, d, e, F2, KK2, 13,  0, x)
        e, b = R(e, a, b, c, d, F2, KK2,  7,  4, x)
        d, a = R(d, e, a, b, c, F2, KK2,  5, 13, x) #/* #47 */
        #/* Parallel round 4 */
        c, e = R(c, d, e, a, b, F1, KK3, 15,  8, x)
        b, d = R(b, c, d, e, a, F1, KK3,  5,  6, x)
        a, c = R(a, b, c, d, e, F1, KK3,  8,  4, x)
        e, b = R(e, a, b, c, d, F1, KK3, 11,  1, x)
        d, a = R(d, e, a, b, c, F1, KK3, 14,  3, x)
        c, e = R(c, d, e, a, b, F1, KK3, 14, 11, x)
        b, d = R(b, c, d, e, a, F1, KK3,  6, 15, x)
        a, c = R(a, b, c, d, e, F1, KK3, 14,  0, x)
        e, b = R(e, a, b, c, d, F1, KK3,  6,  5, x)
        d, a = R(d, e, a, b, c, F1, KK3,  9, 12, x)
        c, e = R(c, d, e, a, b, F1, KK3, 12,  2, x)
        b, d = R(b, c, d, e, a, F1, KK3,  9, 13, x)
        a, c = R(a, b, c, d, e, F1, KK3, 12,  9, x)
        e, b = R(e, a, b, c, d, F1, KK3,  5,  7, x)
        d, a = R(d, e, a, b, c, F1, KK3, 15, 10, x)
        c, e = R(c, d, e, a, b, F1, KK3,  8, 14, x) #/* #63 */
        #/* Parallel round 5 */
        b, d = R(b, c, d, e, a, F0, KK4,  8, 12, x)
        a, c = R(a, b, c, d, e, F0, KK4,  5, 15, x)
        e, b = R(e, a, b, c, d, F0, KK4, 12, 10, x)
        d, a = R(d, e, a, b, c, F0, KK4,  9,  4, x)
        c, e = R(c, d, e, a, b, F0, KK4, 12,  1, x)
        b, d = R(b, c, d, e, a, F0, KK4,  5,  5, x)
        a, c = R(a, b, c, d, e, F0, KK4, 14,  8, x)
        e, b = R(e, a, b, c, d, F0, KK4,  6,  7, x)
        d, a = R(d, e, a, b, c, F0, KK4,  8,  6, x)
        c, e = R(c, d, e, a, b, F0, KK4, 13,  2, x)
        b, d = R(b, c, d, e, a, F0, KK4,  6, 13, x)
        a, c = R(a, b, c, d, e, F0, KK4,  5, 14, x)
        e, b = R(e, a, b, c, d, F0, KK4, 15,  0, x)
        d, a = R(d, e, a, b, c, F0, KK4, 13,  3, x)
        c, e = R(c, d, e, a, b, F0, KK4, 11,  9, x)
        b, d = R(b, c, d, e, a, F0, KK4, 11, 11, x) #/* #79 */

        t = (state[1] + cc + d) % 0x100000000
        state[1] = (state[2] + dd + e) % 0x100000000
        state[2] = (state[3] + ee + a) % 0x100000000
        state[3] = (state[4] + aa + b) % 0x100000000
        state[4] = (state[0] + bb + c) % 0x100000000
        state[0] = t % 0x100000000

    return ripemd160

ripemd160 = gen_ripemd160_with_variable_scope_protector_to_not_pollute_global_namespace()
print(ripemd160(b'hello this is a test').hex())
print("number of bytes in a RIPEMD-160 digest is: ", len(ripemd160(b'')))

f51960af7dd4813a587ab26388ddab3b28d1f7b4
number of bytes in a RIPEMD-160 digest is:  20


The general idea here is similar to SHA256. Again it is a "bit mixer" of a lot of binary operations to obsecure the original message.

Now, we are ready to generate our Bitcoin address. Let's first create a subclass of `Point` called `PublicKey`. 
If we recall correctly, public key is just a Point on the Curve but now has some additional semantics and interpretation of a Bitcoin public
key, together with some methods of encoding/decoding the key into bytes for communication in the Bitcoin protocol.

In [69]:
class PublicKey(Point):
    """
    The public key is just a Point on a Curve, but has some additional 
    specific encoding/decoding functionality that this class implements.
    """
    
    @classmethod
    def from_point(cls, pt: Point):
        """ promote a Point to be a PublicKey"""
        return cls(pt.curve, pt.x, pt.y)
    
    def encode(self, compressed, hash160=False):
        """return the SEC bytes encoding of the public key Point"""
        
        # calculate the bytes
        if compressed:
            # (x,y) is very redundant, why? Because Bicoin curve is
            # y^2 = x^3 + 7, so we can just encode x and then
            # y = +/- sqrt(x^3 + 7), so we need one more bit to encode
            # whether it was the + or - but because this modular arithmetic
            # there is no +/-, instead it can be shown that one y will always
            # be even and the other odd.
            prefix = b'\x02' if self.y % 2 == 0 else b'\x03'
            pkb = prefix + self.x.to_bytes(32, 'big')
        else:
            pkb = b'\x04' + self.x.to_bytes(32, 'big') + self.y.to_bytes(32, 'big')
        # Hash if desired
        return ripemd160(sha256(pkb)) if hash160 else pkb
    
    def address(self, net: str, compressed: bool) -> str:
        """ return the associated bitcoin address for this public key as string """
        # encode the public key into bytes and hash to get their payload
        pkb_hash = self.encode(compressed=compressed, hash160=True)
        # add version byte (0x00 for Main Network, or 0x6f for Test Network)
        version = {'main': b'\x00', 'test': b'\x6f'}
        ver_pkb_hash = version[net] + pkb_hash
        # Calculate the checksum
        checksum = sha256(sha256(ver_pkb_hash))[:4]
        # append to form the full 25-byte binary Bitcoin Address
        # so, the result is composed of three items: a prefix, the data, and a checksum
        byte_address = ver_pkb_hash + checksum
        # finally the result is encoded using Base58 encode.
        b58check_address = b58encode(byte_address)
        return b58check_address

For one moment, I was thinking what this b58encode was or if we defined somewhere. Python indeed has
a package for implementing Base58 encode. But since we want to do everything from scratch here, let's
implement this by defining a function.

In [76]:
# Base58 encoding/decoding utilities
# Reference: https://en.bitcoin.it/wiki/Base58Check_encoding

# alphabet used in Base58 encode, "O"(capital o), "0"(number zero), "I" (capital i), "l" (lower case) are excluded.
alphabet = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'

def b58encode(b: bytes) -> str:
    assert len(b) == 25 # version is 1 byte, pkb_hash 20 bytes, checksum 4 bytes
    n = int.from_bytes(b, 'big')
    chars = []
    while n:
        n, i = divmod(n, 58) # returns quotient and the remainder
        chars.append(alphabet[i])
    # special case handle the leading 0 bytes...
    num_leading_zeros = len(b) - len(b.lstrip(b'\x00'))
    res = num_leading_zeros * alphabet[0] + ''.join(reversed(chars))
    return res

Finally, we are ready to print our Bitcoin address:

In [77]:
# We are going to use "test net" for this exploration since it is the developer's Bitcoin parallel universe
# so, net='test'
address = PublicKey.from_point(public_key).address(net='test', compressed=True)
print(address)

n1zrQKCc6K3sKiFv5hLg8JoPNLPbTZpC9k


---
We can check Bitcoin block explorer to verify if this address has transacted before. For this purpose, we can use
https://www.blockchain.com/btc-testnet/blocks. The [result](https://www.blockchain.com/search?search=n1zrQKCc6K3sKiFv5hLg8JoPNLPbTZpC9k)
shows no transaction associated with this address. This makes sense because there would have to be some other "Aysajan" with a bad sense of humor also tickering with Bitcoin, which is one in a million years I guess. 

Of course, we can play around with some non-secret secret keys and see if anyone used it before. For example, we can check the address associated with the lowest valid secret key of 1, where the public key is exactly the generator point. Let's test this out:

In [79]:
lol_secret_key = 1
lol_public_key = lol_secret_key * G
lol_address = PublicKey.from_point(lol_public_key).address(net='test', compressed=True)
lol_address

'mrCDrCybB6J1vRfbwM5hemdJz73FwDBC8r'

Looking at the result of https://www.blockchain.com/btc-testnet/address/mrCDrCybB6J1vRfbwM5hemdJz73FwDBC8r, we can see that this address has
transacted 1933 times so far with 0 $BTC in the wallet. It makes sense because if it did have any balance then anyone would just be able to spend it because they know the secret key (of 1) and can use it to digitally sign transactions that spend it.

In [80]:
# Let's summarize what we achieved so far:

print("Our first Bitcoin identity:")
print("1. secret key: ", secret_key)
print("2. public key: ", (public_key.x, public_key.y))
print("3. Bitcoin address: ", address)

Our first Bitcoin identity:
1. secret key:  5703626118991814761598925171845964482230864
2. public key:  (114156641397826299630828824355167492942493011232318490144835676357671279887432, 63096451956329221569248690095904719163504830356429270970001086201206174692979)
3. Bitcoin address:  n1zrQKCc6K3sKiFv5hLg8JoPNLPbTZpC9k


---
We zoom out for a bit and think about what steps we went through in order to get this point:

Reference: https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses#How_to_create_Bitcoin_Address

1. Generate a private ECDSA key. How is it done? So we have this special elliptic curve for Bitcoin: $y^2=x^3+7$.
The number of all points on this curve (i.e., the order of the curve) is given by $n$. Then this private key is just a random number 
between 1 (inclusive) and $n$ (exclusive).
2. Genearte public key from the private key. The public key is the __point__ on the curve that results from adding the generator point
to itself prive key times; i.e., 

$$\text{public_key } = G + G + \text{(private_key times)} + G = \text{private_key } \times G$$

The result is again a Point on the curve. Recall that G is the generator point of the curve. This addition of points is implemented by
__Extended Euclidean Algorithm__. 
3. Perform SHA256 hashing on the public key. The main operations involve padding the message, breaking the message into chunks and execute
a number of bit shifts and binary operations to generate fixed-size, random looking value.
4. Perform RIPEMD-160 hashing on the result of SHA256. This is again a number of bit shifts and binary opertations on the output from step 3.
5. Add version byte in front of RIPEMD-160 hash (0x00 for main net).
6. Perform two more SHA256 hashing on the result from step 5.
7. Take the first 4 bytes of the result (which is called checksum) and append it to the address resulted from step 5
8. Convert the result from a byte string into a base58 string using Base58 encoding.

Then there you have it, a Bitcoin address that you can share with any one and publicly transact with.

Andrej provides a shorter layman explanation about all these processes. Basically, we generate a crypto identity 
that consists of a secret key (a random integer within a range) that only we know, and a derived public key by jumping around
the scalar curve using the scalar multiplication of the Generating point (G) on the Bitcoin elliptic curve. We then derive the
associated Bitcoin address which we can share with others. Doing so involves the introduction of two hash functions (SHA256 and RIPEMD160).

## Obtaining Seed Funds

Once we have a crypto identity, we can create a transaction. Our first transaction will be sending some BTC from the address we generated
above (n1zrQKCc6K3sKiFv5hLg8JoPNLPbTZpC9k) to some second wallet we control. So, first let's create this second identity of ours.

In [89]:
# secret_key2 = random.randrange(1, bitcoin_gen.n) # Again this would be the right process to use
secret_key2 = int.from_bytes(b"Super secret 2nd wallet", 'big')
assert 1 <= secret_key2 < bitcoin_gen.n # check if it is valid
public_key2 = secret_key2 * G
address2 = PublicKey.from_point(public_key2).address(net='test', compressed=True)

print("Our second Bitcoin identity: ")
print("1. secret key: ", secret_key2)
print("2. public key: ", (public_key2.x, public_key2.y))
print("3. Bitcoin address", address2)

Our second Bitcoin identity: 
1. secret key:  7993759782619694384297481201601471266087073879975290228
2. public key:  (68950971624828955198341676488534015807203381287337197283541725513678050167539, 20537401902890456163713084398204605639097073431787754138343419103713175529094)
3. Bitcoin address n3MdbGZ9tw8ahJv8YEPAYFKSQrX8MywaoH


---
Now we have our second Bitcoin wallet. What we want to achieve next is to send some BTC from my first address n1zrQKCc6K3sKiFv5hLg8JoPNLPbTZpC9k to second address n3MdbGZ9tw8ahJv8YEPAYFKSQrX8MywaoH. But since we just created both of these addresses, there will not be any BTC in them. So we need to first get some "free" BTC by asking some available faucets. Of course those BTC have no value since we are playing with test network, not the Main Network. Test network is the "parallel universe" developer-intended Bitcoin network. I just googled "bitcoin testnet faucet" and the first result came out was https://coinfaucet.eu/en/btc-testnet/. I used this website to
get my BTC for the first address. [The result](https://www.blockchain.com/btc-testnet/address/n1zrQKCc6K3sKiFv5hLg8JoPNLPbTZpC9k) shows that I now have $0.01490097 BTC in my first wallet.

Next we explore the transaction details. If we click the exact [transaciton ID](https://www.blockchain.com/btc-testnet/tx/b60c6b4fca235abd0990f275c2dbeccec479864d597afc1c89e1515257c45342) we can see various information that gets to the heart of the Bitcoin
and how money is represented in it. Let's take a closer look at several components.

- Transaction ID, or Hash. Every transaciton has a distinct id/hash. For our transaction the hash was _b60c6b4fca235abd0990f275c2dbeccec479864d597afc1c89e1515257c45342_. From its name, one can guess that it is a hash value. More precisely,
this is just SHA256 double hash (hash of a hash) of the transaction data structure. In Bitcoin double SHA256 hashes are used quite often
instead of a single hash for added security. But I am not quite sure what security needs to be added for a transaction id. 
- Inputs and Outputs. As we can see, the faucet transaction has one input and two outputs. The input came from address _tb1qyshspra4yeetah4c3m2c2lje3ffzfe3yq4k92m_ of value 28.22958907 BTC. There were two outputs and first one is associated with our address. 
Our address received 0.01490097 BTC. The address in the second output is unknown and it received 28.21453860 BTC. We can do a quick math here:

$$28.22958907 - (0.01490097+28.21453860)=0.00014950$$

This "change" amount is called the fee, claimed by the Bitcoin miner who has included this transaction in their block, which in our case is
[Block 2286951](https://www.blockchain.com/btc-testnet/block/2286951). 99 transactions were included in this block and we can find our
transaction [there](https://www.blockchain.com/btc-testnet/block/000000005f51b7fc6ea11206589860ef35b7576060f68e9589d4fdb3d47e0a6f?page=5).
So, this fee acts as a financial incentive for miners to include this transaction in this block, i.e., finalize the transaction. You can guess
by now: the higher the fee to the miner, the more likely and faster the transaction to appear in the blockchain.

There is one special type of transaction in this block, which appears as the first transaction. One special aspect of this transaction is that
there is no input. So basically the miner is sending BTC from a null input to themselves, and the total amount is 0.05330744 BTC. It is the sum of two types of rewards the miner gets:
1. Block Reward. This is the incentive for this specific miner with the address 2NEeyjxe8VANDqBkbduhcr5Lzxw4LTujNCu for doing all the computation work and mining a new block. The block reward for this specific block is 0.04882812 BTC 
2. Fee Rewrad. And then there is second reward of 0.00447932 BTC, which is the sum of all transaction fees paid by the users whose transactions
were included in this block by the miner.

- Size. Also note that this transaction was 228 bytes.
- Pkscript. We also note that the output details has a "Pkscript" field, which has:

`OP_DUP`

`OP_HASH160`

`e0a93e8171c874d2688f113290ce425fdbe46969`

`OP_EQUALVERIFY`

`OP_CHECKSIG`

You might wonder where the hell the code is. Well, Bitcoin indeed has a whole stack-based scripting language. But unless you are doing some
crazy things with Bitcoin network, a few simple scripts take care of most of the transactions. These Pkscript just represents a list of
scripts used in this transaction. This `Pkscript` is the "locking script" for this specific Output, which holds our 0.01490097 BTC.

Apparently one obvious thing we might do is to spend this Output and turn it into an Input in our next transaction. In order to unlock this
output, we need to satisfy the conditions of this locking script. In layman's term, this script is saying that any transaction that aspires to spend this Output must satisfy two conditions:
1) Their public key better hash to e0a93e8171c874d2688f113290ce425fdbe46969, and
2) The digital signature for the aspiring transaction better validate as being generated by this public key's associated private key.

Obviously, only the real owner of the secret key can do both.

But first, let's try to verify that our public key indeed hashes to the digest specified in this output of our faucet transaction. If they
match, that means we satisfy first condition. As we can see from the examination below, they indeed match. Sweet. No problem with the
first condition of spending this BTC.

In [94]:
PublicKey.from_point(public_key).encode(compressed=True, hash160=True).hex() == 'e0a93e8171c874d2688f113290ce425fdbe46969'

True

Now let's focus on some other transactions we can do.

## Crafting Transaction

Okay. Let's play with it a little bit more. What we want to try out next is to send half of our funds in our first address to our second wallet.
To achieve this , as before our transaction will have exactly one input (= 1st output of the previous transaction, i.e., faucet transaction) 
and exactly two outputs. One output will go to our 2nd address, the rest of it will send back to our 1st address. 

So you might wonder, why we just have one input and one output where half of the BTC in my 1st wallet be send directly to second wallet.
The answer is that in Bitcoin nextwork, Every Input/Output of any bitcoin transaciton must always be fully spent. So if I own 1 BTC and
want to send half of it to someone else, I actually have to send one half there and one half to myself. The transaction will be considered
valid if the sum of all outputs < the sum of all inputs. The remainder will be the "change" that the winning miner who successfully mined a 
new block by doing all the work will keep.

Let's play around with it next.

In [98]:
@dataclass
class TxIn:
    prev_tx: bytes # previous transaction ID: hash256 of prev tx contents
    prev_index: int # UTXO output index in the transaction
    script_sig: Script = None # unlocking script
    sequence: int = 0xffffffff # originally intended for "high frequency trades", with locktime
    
tx_in = TxIn(
    prev_tx = bytes.fromhex('b60c6b4fca235abd0990f275c2dbeccec479864d597afc1c89e1515257c45342'),
    prev_index = 0,
    script_sig = None, # this field will have digital signature, to be inserted later
)

The first two variables `prev_tx` and `prev_index` identify the specific Output that we are going to spend. Note that nowhere are we specifying how much of the output we want to spend because we have to spend the entire output (a "UTXO" short for "Unspent Transaction Output"). Once we consume the entire UTXO, we are free to "chunk up" its values however many outputs we like. For the index part, we want to spend the
amount from the first output with index 0 because that's the output associated with our own address. The second output
is for some unknown address that we don'a have control to.

`script_sig` is the field that the digital signature will go, cryptographically signing the desired transaction with out private key and
effectively saying "I approve this transaction as the possessor of the private key whose public key hashes to e0a93e8171c874d2688f113290ce425fdbe46969".

`sequence` was originally implemented by Satoshi but has very limited uses today and we just ignore it.

__Calculate the fee.__ So, our data structure defined above references Inputs of our next transaction (1 input). Next, let's create data structure for two outputs. As a first setp, let's calculate the tip that we are going to give to the winning miner. In practice, users rarely
specify the tips themselves and software wallets will automatically include best current rates into our transaction. Since here we
are doing everything from scratch, let's calculate how much we plan to tip. In order to do that, we can either check some "market rate"
available on some websites or we can look at the transactions in the block that has our transaction and get a rough estimate of the current
market rate. It seems like the fee is around 65 satoshi/byte. So, let's try to go with a relatively generous fee of 100 satoshi/byte.
If you recall, the size of our transaction was 228 bytes. Let's round it to 250 bytes, so the fee will be 25,000 satoshi. 
We have 0.01490097 BTC which is equivalent to 1,490,097 satoshi. Let's send 750,000 satoshi to our second address and the rest
`(1,490,097 - 25,000 - 750,000) = 715,097` satoshi back to us.

In [99]:
@dataclass
class TxOut:
    amount: int # in units of satoshis (1e-8 of bitcoin)
    script_pubkey: Script = None # locking script
    
tx_out1 = TxOut(
    amount = 750000 # As in the "Calculating the fee", we will send this 750,000 sat to our target address
)
tx_out2 = TxOut(
    amount = 715097 # back to us
)
# the fee of 25,000 sat does not need to be manually apecified, the miner will claim it

---
__Populating the locking scripts__. Next, we populate the `script_pubkey` "locking script" for both of these outputs. What we want to achieve under the hood is to specify the conditions under which each output can be spent by some future transactions. To indicate the ownership of both of these outputs we basically want to specify the public key hash of whoever can spend the output. Except we have to dress that up with the 
"rich scripting language" padding. Ok here we go.

If we recall the locking script in the transaction output, when we checked block explorer to see the transaction details, the public key hash of the owner of each output was sandwiched between a few Bitcoin Scripting Language op codes:

`OP_DUP`

`OP_HASH160`

`e0a93e8171c874d2688f113290ce425fdbe46969`

`OP_EQUALVERIFY`

`OP_CHECKSIG`

Let's try to understand these several special-purpose instructions in bitcoin script language (Reference: [Bitcoin and CryptoCurrency Technologies](https://www.amazon.ca/Bitcoin-Cryptocurrency-Technologies-Comprehensive-Introduction/dp/0691171696)). First of all, the scripting language is
stack-based. What that means is that each insturction is executed exactly once, in a linear manner. In particular, there are no
loops in the Bitcoin scripting language. 
1. First we have the duplicate instruction, `OP_DUP`. What it does is to push a copy of the piblic key onto the top of the stack.
2. Next instruction is `OP_HASH160`, which tells us to pop the top stack value, compute its cryptographic hash, and push the result
onto the top of the stack. It hashes twice: first using SHA256 and then RIPEMD160.

When this instruction finishes executing, we have replaced the public key on top of the stack with its hash.

3. We do one more push of data on the stack. Recall that this data was specified by the sender of the referenced transaction. It is the hash
of the public key that the sender specified (for instance, I specify the hash of the public key of the recipients in my outgoing 
tx.). The corresponding private key must be used to generate the digital signature to redeem these coins. At this point we have two values
at the top of the stack: the hash of the public key (specified by the sender) and the hash of the public key that would be used by the recipient when trying to claim the coins.

4. Next, `OP_EQUALVERIFY` instruction executes, which, you might guess, checks these two hashes at the top of the stack are equal. 
If not, an error will be thrown. This operation will consume two values at the top and the stack now only has two items: a signature and the public key.

Now, we have already checked that the publick key is in fact the public key specified in the transaction. We still need to do one more
check, which is signature. We need to check the signature is valid.

5. Instruction `OP_CHECKSIG` does the entire signature verification. The logic is that this instruction uses public key to verify the signature is valid for the entire transaciton.

Once all these instructions are executed, there is nothing left on the stack. Provided no errors occurred, the output of this script will be `true`, indicating that the transaction is valid.

---
We need to create this same structure and encode it into bytes, but we want to swap out the public key hash with the new owner's hashes

In [101]:
def encode_int(i, nbytes, encoding='little'):
    """ encode integer i into given number of bytes using a given byte ordering """
    return i.to_bytes(nbytes, encoding)

def encode_variant(i):
    """ encode a (possible but rarely large) integer into bytes with a super simple compression scheme """
    if i < 0xfd:
        return bytes([i])
    elif i < 0x10000:
        return b'\xfd' + encode_int(i, 2)
    elif i < 0x100000000:
        return b'\xfe' + encode_int(i, 4)
    elif i < 0x10000000000000000:
        return b'\xff' + encode_int(i, 8)
    else:
        raise ValueError("integer too large: %d" % (i, ))
        
@dataclass
class Script:
    cmds: List[Union[int, bytes]]
    
    def encode(self):
        out = []
        for cmd in self.cmds:
            if isinstance(cmd, int):
                # an int is just an opcode, encode as a single byte
                out += [encode_int(cmd, 1)]
            elif isinstance(cmd, bytes):
                # bytes represent an element, encode its length and then content
                length = len(cmd)
                assert length < 75 # any longer than this requires some tedious handling that we will skip here
                out += [encode_int(length, 1), cmd]
        
        ret = b''.join(out)
        return encode_variant(len(ret)) + ret
    
# The first output will go to our second wallet
out1_pkb_hash = PublicKey.from_point(public_key2).encode(compressed=True, hash160=True)
out1_script = Script([118, 169, out1_pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG
print(out1_script.encode().hex())

# teh second output will go back to us
out2_pkb_hash = PublicKey.from_point(public_key).encode(compressed=True, hash160=True)
out2_script = Script([118, 169, out2_pkb_hash, 136, 172])
print(out2_script.encode().hex())

1976a914ef8f69340792be772a65b8a094ed0188f454688788ac
1976a914e0a93e8171c874d2688f113290ce425fdbe4696988ac


For those who might wonder about those four numbers (118, 169, 136, and 172), these are the opcode for script instructions. 
For instance, Opcode for `OP_DUP` is 118 and 172 for `CHECKSIG`.

Reference: https://en.bitcoin.it/wiki/Script

Okay, now we are ready to officially declare the owners of both outputs of our transaction 
by specifying the public key hashes (padded by the Script Opcodes). We'll see how these locking scripts work for the Outputs when we create
the unlocking script for the Input. Basically, when the unlocking happens for inputs, transaction happens and Output is generated, which will
be followed by a locking scripts for the Outputs. For now, try to understand that we are effectively declaring the owner of each output UTXO
by identifying a specific public key hash. With the locking script specified as above, only the person who has the original public key 
(and its associated secret key) will be able to spend the UTXO.

In [103]:
tx_out1.script_pubkey = out1_script
tx_out2.script_pubkey = out2_script

## Digital Signature

If we look at the transaction details for our [faucet transaction](https://www.blockchain.com/btc-testnet/tx/b60c6b4fca235abd0990f275c2dbeccec479864d597afc1c89e1515257c45342), we can see a field in the Inputs called `Sigscript`. Here what we
want to accomplish is that we want to create a digital signature that effectively says "_I, the owner of the private key associated with the
public key hash on the referenced transaction's output's locking script approve the spend of this UTXO as an input of this transaction._"

This is where Bitcoin gets pretty fancy and it turns out you can only sign one thing in Bitcoin - an entire transaction. 
You can actually only sign parts of Transactions, a dn a number of signatures can be assembled from a number of parties and combined in
various ways. But Bitcoin says "No, I will just sign once on the entire transaciton." We next construct the unlocking script specifically
to only specify the locking script of the exact form above (OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG).
    
Coouple of things we need to do:
    1. We need to create a pure bytes "message" what we will be digitally signing. In this case, the message is the encoding of
    the entire transaction. This is weird because we haven't finished encoding the entire transaction into bytes because we are still missing
    our signature. This is what we are currently working on. What do we do? The rule is to replace the encoding of the `script_sig` with the
    `script_pubkey` of the transaction output this input is pointing back to. All other transaction input's `script_sig` is also replaced
    with an empty script because I just need to contribute my own signature and others will contribute their own signatures in their
    script.
    
    We need the final data structure, the actual Transaction, in order to encode the entire transaction into the bytes message. It will be
    a container for a list of `TxIn`'s and list of `TxOut`'s: the inputs and outputs. We then implement the serialization for the new `Tx` class, and also the serialization of `TxIn` and `TxOut` class, so we can serialize the entire transaction to bytes.
    
---
Wow, that is a lot. Let's see the code in action.

In [114]:
@dataclass
class Tx:
    version: int
    tx_ins: List[TxIn]
    tx_outs: List[TxOut]
    locktime: int = 0
    
    def encode(self, sig_index=-1) -> bytes:
        """
        Encode this transaction as bytes.
        If sig_index is given then return the modified transaction 
        encoding of this tx with respect to the single input index.
        This result then constitutes the "message" that gets signed
        by the aspiring transactor of this input.
        """
        out = []
        # encode metadata
        out += [encode_int(self.version, 4)]
        # encode inputs
        out += [encode_variant(len(self.tx_ins))]
        if sig_index == -1:
            # we are just serializing a fully formed transaction
            out += [tx_in.encode() for tx_in in self.tx_ins]
        else:
            # used when crafting digital signature for a specific input index
            out += [tx_in.encode(script_override=(sig_index == i)) 
                    for i, tx_in in enumerate(self.tx_ins)]

        # encode outputs
        out += [encode_variant(len(self.tx_outs))]
        out += [tx_out.encode() for tx_out in self.tx_outs]
        # encode other metadata
        out += [encode_int(self.locktime, 4)]
        out += [encode_int(1, 4) if sig_index != -1 else b''] # 1 = SIGHASH_ALL
        return b''.join(out)

# We also need to know how to encode TxIn. This is just serialization protocol.
def txin_encode(self, script_override=None):
    out = []
    out += [self.prev_tx[::-1]] # little endian vs big endian encodings
    out += [encode_int(self.prev_index, 4)]
    
    # three scenarios for signature replacement
    if script_override is None:
        # None = just sue the actual script
        out += [self.script_sig.encode()]
    elif script_override is True:
        # True = override the script with the script_pubkey of the associated input
        out += [self.prev_tx_script_pubkey.encode()]
    elif script_override is False:
        # False = override with an empty script
        out += [Script([]).encode()]
    else:
        raise ValueError("script_override must be one of None|True|False")
        
    out += [encode_int(self.sequence, 4)]
    return b''.join(out)

TxIn.encode = txin_encode # monkey patch into the class

# and TxOut as well
def txout_encode(self):
    out = []
    out += [encode_int(self.amount, 8)]
    out += [self.script_pubkey.encode()]
    return b''.join(out)

TxOut.encode = txout_encode # monkey patch into the class

tx = Tx(
    version = 1,
    tx_ins = [tx_in],
    tx_outs = [tx_out1, tx_out2],
)

We need to recreate the data structure of the first output (associated with our first address) as a script since the Block Explorer website
does not allow us to get this in the bytes form. Again, the reason why we want to do is that we need to sign the entire transaction but 
we don't have `script_sig`. So we replace it with the public key hash of the output from previous transaction. [Here](https://www.blockchain.com/btc-testnet/tx/b60c6b4fca235abd0990f275c2dbeccec479864d597afc1c89e1515257c45342) is the link for our very first faucet transaction that we are referring to. In our case, we are trying to spend its Output at Index 0, and the script_pubkey is, again

`OP_DUP`

`OP_HASH160`

`e0a93e8171c874d2688f113290ce425fdbe46969`

`OP_EQUALVERIFY`

`OP_CHECKSIG`

We will re-create the data structures as a Script:

In [115]:
source_script = Script([118, 169, out2_pkb_hash, 136, 172])
print("recall out2_pkb_hash is just raw bytes of the hash of public_key: ", out2_pkb_hash.hex())
print(source_script.encode().hex()) # we can get the bytes of the script_pubkey now

recall out2_pkb_hash is just raw bytes of the hash of public_key:  e0a93e8171c874d2688f113290ce425fdbe46969
1976a914e0a93e8171c874d2688f113290ce425fdbe4696988ac


In [116]:
# monkey patch this into the input of the transaction we are trying to sign and construct
tx_in.prev_tx_script_pubkey = source_script

# get the "message" we need to digitally sign
message = tx.encode(sig_index = 0)
message.hex()

'01000000014253c4575251e1891cfc7a594d8679c4ceecdbc275f29009bd5a23ca4f6b0cb6000000001976a914e0a93e8171c874d2688f113290ce425fdbe4696988acffffffff02b0710b00000000001976a914ef8f69340792be772a65b8a094ed0188f454688788ac59e90a00000000001976a914e0a93e8171c874d2688f113290ce425fdbe4696988ac0000000001000000'

Okay, the term "monkey patch" came up multiple times and at first I thought it is just a word for fun. But later it seemed like an actual
operations. And yes, indeed, it is an actual programming terminology. [The definition](https://en.wikipedia.org/wiki/Monkey_patch) 
from wikipedia says that "A monkey patch is a way for a program to extend or modify supporting system software locally (affecting only the
running instance of the program)." The below Python example monkey-patches the value of `Pi` from standard Python math library to make it
compliant with the India Pi Bill.

In [117]:
import math
print(math.pi)
math.pi = 3.2 # monkey-patch the value of Pi in the math module
math.pi

3.141592653589793


3.2

In [120]:
# After we restart and run the code below again, Pi value will go back to 3.1415926...
import math
math.pi

3.2

---
Okay, enough of a detour. Let's pause a moment. There are lots of things to digest here. First of all, what did we do? we have encoded the transaction into bytes to create a "message", in the digital signature lingo. Let's think about what the above bytes encode, and what it is that we are about to sign. Couple of things we did in the process:
- We identify the exact inputs of this transaction by referencing the outputs of a specific previous transactions.
- We also identify the exact outputs of this very specific transaction (not yet executed, it is newly about to be minted UTXOs, so to speak) - - We also identify their `script_pubkey` fields, which in the most common case declare an owner of each output via their public key hash wrapped up in a script.

So, what this message really encodes is the inputs and the new outputs, their amounts, and their owners (via locking scripts specifying the public key hash of each owner.)

---
Great. We are now ready to digitally sign the message with out private key. The actual signature itself is a tuple of two integers `(r, s)`. As with Elliptic Curve Cryptography (ECC) above, we don't cover the full mathematical details of the Elliptic Curve Digital Signature Algorithm (ECDSA). Instead we use the code to understand the basic mechanism of it. As we know it, digital signarure consists of three components:
- private/public key pair generation, which we have already done.
- sign the message to generate a signature.
- verify the signature.

The last two components are implemented below.

In [122]:
@dataclass
class Signature:
    r: int
    s: int
    
def sign(secret_key: int, message: bytes) -> Signature:
    # the order of the elliptic curve used in bitcoin
    n = bitcoin_gen.n
    
    # double hash the message and convert to integer
    z = int.from_bytes(sha256(sha256(message)), 'big')
    
    # generate a new secret/public key pair at random
    sk = random.randrange(1, n)
    P = sk * bitcoin_gen.G
    
    # Calculate the signature
    r = P.x
    s = inv(sk, n) * (z + secret_key * r) % n
    if s > n / 2:
        s = n - s
    
    sig = Signature(r, s)
    return sig

def verify(public_key: Point, message: bytes, sig: Signature) -> bool:
    # just a stub for reference on how a signature would be verified in terms of the API
    # we don't need to verify any signatures to craft a transaction, but we would if we were mining
    pass

random.seed(int.from_bytes(sha256(message), 'big')) # see note below
sig = sign(secret_key, message)
sig

Signature(r=6490053236317912098198246005738199062818346316985474975194943187922392493408, s=55151421285902115870196508307045006518952914335579462009163427116513860522215)

---
Let's now implement the `encode` function of a `Signature` so we can broadcast it over the Bitcoin protocol. Basically, an ECDSA signature
passed to various instructions in Bitcoin Script must be encoded using strict [DER Encdoding](https://en.bitcoin.it/wiki/BIP_0062#DER_encoding). We cannot just use the raw signature generated by our Signature algorithm. Bitcoin involved with various encoding
and hashing almost eerywhere.

In [123]:
def signature_encode(self) -> bytes:
    """ return the DER encoding of this signature"""
    
    def dern(n):
        nb = n.to_bytes(32, byteorder='big')
        nb = nb.lstrip(b'\x00') # strip leading zeros
        nb = (b'\x00' if nb[0] >= 0x80 else b'') + nb # prepend 0x00 if first byte >= 0x80
        return nb
    rb = dern(self.r)
    sb = dern(self.s)
    content = b''.join([bytes([0x02, len(rb)]), rb, bytes([0x02, len(sb)]), sb])
    frame = b''.join([bytes([0x30, len(content)]), content])
    return frame

Signature.encode = signature_encode # again, monkey patch into the class
sig_bytes = sig.encode()
sig_bytes.hex()

'304402200e593d6dd30b5b5fb948803a8fd64a938862eb64b961a8ca75616c4a8dccad60022079ee99268738f81621108256f103863327c6a588ba1a515286041ce444a810e7'

Bear with me. We are finally ready to generate the `script_sig` for the single input of our transaction. It will contain exactly two elements: 

1) the signature and
2) the public key.

It will become clear in a moment why we need these two elements.

In [124]:
# Append 1 (= SIGHASH_ALL), indicating this DER signature we created encode 'ALL' of the tx (by far most common)
sig_bytes_and_type = sig_bytes + b'\x01'

# Encode the public key into bytes. Notice we use hash160=False so we are revealing the full public key to Blockchain
pubkey_bytes = PublicKey.from_point(public_key).encode(compressed=True, hash160=False)

# Create a lightweight Script that just encoded those two things
script_sig = Script([sig_bytes_and_type, pubkey_bytes])
tx_in.script_sig = script_sig

Now we created both locking scripts ('script_pubkey') and the unlocking scirpts (`script_sig`). 
Let's reflect how these two scripts interact in the Bitcoin scripting environmenet. On a high level, in the transaction validating process
during mining, for each transaction input the two scripts get concatenated into a single script. We can see now that concatenating
the two scripts will look like:

`<sig_bytes_and_type>`

`<pubkey_bytes>`

`OP_DUP`

`OP_HASH160`

`<pubkey_hash_bytes>`

`OP_EQUALVERIFY`

`OP_CHECKSIG`

This then gets executed top to bottom with a typical stack-based push/pop scheme, where any bytes get pushed into the stack, and any ops will
consume some inputs and push some outputs. Several steps in order:

- We push to the stack the signature and the pubkey.
- The pubkey gets duplicated (OP_DUP).
- The pubkey gets hashed (OP_HASH160).
- The hash gets compared to the `pubkey_hash_bytes` (OP_EQUALVERIFY).
- Finally the digital signature integrity is verified as having been signed by the associated private key.

---
OKAYYYYYYYYYYY. We have now have all the necessary steps! Let's take a look at a repr of our fully constructed transaction again:

In [125]:
tx

Tx(version=1, tx_ins=[TxIn(prev_tx=b'\xb6\x0ckO\xca#Z\xbd\t\x90\xf2u\xc2\xdb\xec\xce\xc4y\x86MYz\xfc\x1c\x89\xe1QRW\xc4SB', prev_index=0, script_sig=Script(cmds=[b"0D\x02 \x0eY=m\xd3\x0b[_\xb9H\x80:\x8f\xd6J\x93\x88b\xebd\xb9a\xa8\xcaualJ\x8d\xcc\xad`\x02 y\xee\x99&\x878\xf8\x16!\x10\x82V\xf1\x03\x863'\xc6\xa5\x88\xba\x1aQR\x86\x04\x1c\xe4D\xa8\x10\xe7\x01", b'\x03\xfcb^\x8c%w\x9a9\x9a\xce{\x8f\xbc\x0c\x03p\xee\x886\xae\x9cV\xa4\xc9\xde\xa3\xd0\x06\x80\xeb\x84H']), sequence=4294967295)], tx_outs=[TxOut(amount=750000, script_pubkey=Script(cmds=[118, 169, b'\xef\x8fi4\x07\x92\xbew*e\xb8\xa0\x94\xed\x01\x88\xf4Th\x87', 136, 172])), TxOut(amount=715097, script_pubkey=Script(cmds=[118, 169, b'\xe0\xa9>\x81q\xc8t\xd2h\x8f\x112\x90\xceB_\xdb\xe4ii', 136, 172]))], locktime=0)

As you can see, there is not that much to a Bitcoin transaction. Let's encode it (AGAIN!!) into bytes and show in hex:

In [126]:
tx.encode().hex()

'01000000014253c4575251e1891cfc7a594d8679c4ceecdbc275f29009bd5a23ca4f6b0cb6000000006a47304402200e593d6dd30b5b5fb948803a8fd64a938862eb64b961a8ca75616c4a8dccad60022079ee99268738f81621108256f103863327c6a588ba1a515286041ce444a810e7012103fc625e8c25779a399ace7b8fbc0c0370ee8836ae9c56a4c9dea3d00680eb8448ffffffff02b0710b00000000001976a914ef8f69340792be772a65b8a094ed0188f454688788ac59e90a00000000001976a914e0a93e8171c874d2688f113290ce425fdbe4696988ac00000000'

In [127]:
print("Transaction size in bytes: ", len(tx.encode()))

Transaction size in bytes:  225


Finally, let's calculate the id of our finished transaction:

In [130]:
def tx_id(self) -> str:
    return sha256(sha256(self.encode()))[::-1].hex() # little/big endian conventions require byte order swap
Tx.id = tx_id # monkey patch into the class

tx.id() # once this transaction goes through, this will be its id

'1ef85a75683cca2fc62983e46b10f1d634f345abe950bae97c4a326f1506a659'

---
At last, we are ready to broadcast our transaction to Bitcoin nodes around the world. We're literally blasting out the 225 bytes (embedded into a standard Bitcoin protocol network envelope) that define our transaction. The Bitcoin nodes will decode it, validate it, and include it into the next block thet mine any second now. In English, those 225 bytes are saying "hello Bitcoin network, how are you? Great. I woould like to create a new transaction that takes the output (UTXO) of the transaction _b60c6b4fca235abd0990f275c2dbeccec479864d597afc1c89e1515257c45342_ at index 0, and I would like to chunk its amount into two outputs, one going to the address n3MdbGZ9tw8ahJv8YEPAYFKSQrX8MywaoH for the amount of 750,000 sat and the rest going to the address n1zrQKCc6K3sKiFv5hLg8JoPNLPbTZpC9k for the amount of 715,097 sat. (It is understood that the rest of 25,000 sat will go to any miner who includes this transaction in their block). Here are two pieces of documentation providing that I can spend this UTXO: my public key, and the digital signature generated by the associated private key, of the above letter of intent. Kkthx!"

---
Next step is to broadcast this out to the network and see if it sticks! We could include a simple client here that speaks the Bitcoin protocol over [`socket`](https://www.ibm.com/docs/en/zos/2.1.0?topic=services-what-is-socket) to communicate to the nodes - we'd first do the handshake (sending versions back and forth) and then broadcast the transaction bytes above using the `tx` message. However, the code is long and not that interesting. So we will instead use blockstream's helpful [tx/push](https://blockstream.info/testnet/tx/push) endpoint to broadcast the transaction. It's just a large textbox where we can copy paste our raw message hex exactly as above, and hit 'Broadcast'. If we'd like to do this manually with raw Bitcoin protocol, Andrej actually wrote a [SimpleNode](https://github.com/karpathy/cryptos/blob/main/cryptos/network.py) implementation and use that to communicate to a node over socket. But again here we just take the short cut.

Okay then, let's broadcast.

In [136]:
import time
time.sleep(1.0) # now we wait, for the network to execute the transaction and include it in a block

Okay! [Here](https://www.blockchain.com/btc-testnet/tx/1ef85a75683cca2fc62983e46b10f1d634f345abe950bae97c4a326f1506a659) it is, our transaction. The network successfully decode our network and reflect all the paratemeter setup correctly. 

After some time (about 20 minutes passed since our broadcast), we can see that the transaction was confirmed and included in the block [2287071](https://www.blockchain.com/btc-testnet/block/2287071). This block includes 96 transactions and ours is one of them. We can also check the UTXO in both of our addresses:
- The [first address](https://www.blockchain.com/btc-testnet/address/n1zrQKCc6K3sKiFv5hLg8JoPNLPbTZpC9k) should have a final balance of 0.00715097 BTC (715,097 sat). Yes, indeed, it has exactly that amount of balance.
- The [second address](https://www.blockchain.com/btc-testnet/address/n3MdbGZ9tw8ahJv8YEPAYFKSQrX8MywaoH) should have a final balance of 0.00750000 BTC (750,000 sat we sent). Yes indeed!

## Putting it all together: One more transaction

We do one last transaction where we create a third identity and cosolidate all of our remaining funds in this one wallet.

In [135]:
secret_key3 = int.from_bytes(b"super secret 3rd wallet", 'big') # or just random.randrange(1, bitcoin_gen.n)
assert 1 <= secret_key3 < bitcoin_gen.n
public_key3 = secret_key3 * G
address3 = PublicKey.from_point(public_key3).address(net='test', compressed=True)

print("Our third Bitcoin identity: ")
print("1. secret key: ", secret_key3)
print("2. public key: ", public_key3)
print("3. Bitcoin address: ", address3)

Our third Bitcoin identity: 
1. secret key:  11058750864351472101014175255902094429478011288825914740
2. public key:  Point(curve=Curve(p=115792089237316195423570985008687907853269984665640564039457584007908834671663, a=0, b=7), x=97858751222056926777078573962560786698485894618880413980373189893650674640395, y=73318358187036259420899005894955755140809196002468074087856377234611185481964)
3. Bitcoin address:  mmQLYiNRkMREqaSs7Sh5TDA8safWGHtmuN


---
As mentioned above, we currently have 715,097 sat in out first wallet n1zrQKCc6K3sKiFv5hLg8JoPNLPbTZpC9k and 750,000 sat in our second wallet n3MdbGZ9tw8ahJv8YEPAYFKSQrX8MywaoH. What we want to do next is to create a transaction with these two inputs, and a single output into the third wallet mmQLYiNRkMREqaSs7Sh5TDA8safWGHtmuN we just created. As before, we will pay 10,000 sat this time as a fee, so we are sending ourselves $715,097 + 750,000 - 10,000 = 1,455,097 $ sat.

In this process, we will try to further understand the steps of creating a Bitcoin transaction and confirm it on the Bitcoin network.

In [142]:
# --------------------------------------------------------
# Keep in mind that we have two inputs and one input in this transaction
# first input of the transaction
tx_in1 = TxIn(
    prev_tx = bytes.fromhex('1ef85a75683cca2fc62983e46b10f1d634f345abe950bae97c4a326f1506a659'),
    prev_index = 0,
    script_sig = None, # digital signature to be inserted later
)
# reconstruct the script_pubkey locking this UTXO (note: it's the first output index in the
# referenced transaction, but the owner is the second crypto identity/wallet we created earlier!)
# recall this information is "swapped in" when we digitally sign the spend of this UTXO a bit later
pkb_hash = PublicKey.from_point(public_key2).encode(compressed=True, hash160=True)
tx_in1.prev_tx_script_pubkey = Script([118, 169, pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG

# -------------------------------------------------------
# second input of the transaction
tx_in2 = TxIn(
    prev_tx = bytes.fromhex('1ef85a75683cca2fc62983e46b10f1d634f345abe950bae97c4a326f1506a659'),
    prev_index = 1,
    script_sig = None, # digital signature to be inserted later
)
pkb_hash = PublicKey.from_point(public_key).encode(compressed=True, hash160=True)
tx_in2.prev_tx_script_pubkey = Script([118, 169, pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG

# ------------------------------------------------------
# Define single output
tx_out = TxOut(
    amount = 1455097,
    script_pubkey = None, # locking script, inserted separately right below
)
# declare the owner as identity 3 we created above, by inserting the public key hash into the Script "padding"
out_pkb_hash = PublicKey.from_point(public_key3).encode(compressed=True, hash160=True)
out_script = Script([118, 169, out_pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG
tx_out.script_pubkey = out_script

# ------------------------------------------------------
# create the aspiring transaction object
tx = Tx(
    version = 1,
    tx_ins = [tx_in1, tx_in2], # 2 inputs this time.
    tx_outs = [tx_out], # a single output
)

# -----------------------------------------------------
# digitally sign the spend of the second input of this transaction
# note that index 0 of the input is our second identity, so it must sign here
message1 = tx.encode(sig_index = 0)
random.seed(int.from_bytes(sha256(message1), 'big'))
sig1 = sign(secret_key2, message1) # identity 2 signs
sig_bytes_and_type1 = sig1.encode() + b'\x01' # DER signature + SIGHASH_ALL
pubkey_bytes = PublicKey.from_point(public_key2).encode(compressed=True, hash160=False)
script_sig1 = Script([sig_bytes_and_type1, pubkey_bytes])
tx_in1.script_sig = script_sig1

# -----------------------------------------------------
# digitally sign the spend of the second input of this transaction
# note that index 1 of the input transaction is our first identity, so it signs here
message2 = tx.encode(sig_index = 1)
random.seed(int.from_bytes(sha256(message2), 'big'))
sig2 = sign(secret_key, message2) # identity 1 signs
sig_bytes_and_type2 = sig2.encode() + b'\x01' # DER signature + SIGHASH_ALL
pubkey_bytes = PublicKey.from_point(public_key).encode(compressed=True, hash160=False)
script_sig2 = Script([sig_bytes_and_type2, pubkey_bytes])
tx_in2.script_sig = script_sig2

# and that should be it!
print(tx.id())
print(tx)
print('*'*100)
print(tx.encode().hex())

8094f5cd6176dc0cc8f61575c9e6cf13b5f5ebd7a78f1eee70719fc0eb26fbf9
Tx(version=1, tx_ins=[TxIn(prev_tx=b'\x1e\xf8Zuh<\xca/\xc6)\x83\xe4k\x10\xf1\xd64\xf3E\xab\xe9P\xba\xe9|J2o\x15\x06\xa6Y', prev_index=0, script_sig=Script(cmds=[b'0E\x02!\x00\xb1\xeeFm\x1b\x12\xf4&\xdc\xb6\xfaU\\I\nnI\x9f\t\xe92\x11\x7fQ+\xdd&\x9a\xea\x0c\x91\xfd\x02 }\x02\x04;3\xb5\x81=N0\xdc\xc8\x9c\xde\x0b*^!1`\xef\xba7\xa9\x8bL\x82\x83\x1f<\xb3\xee\x01', b'\x02\x98p\xdd\xf0w\x86a\xd0\x03\x1e\x80\x90I\x8aa\x16MJ\x0eh\xf2\xcc\xcc9\xf1\x8fc\xf8\x8a\x12\n\xf3']), sequence=4294967295), TxIn(prev_tx=b'\x1e\xf8Zuh<\xca/\xc6)\x83\xe4k\x10\xf1\xd64\xf3E\xab\xe9P\xba\xe9|J2o\x15\x06\xa6Y', prev_index=1, script_sig=Script(cmds=[b"0E\x02!\x00\xea\xfe\x02f\x7f'\x16oy\x11\x82\xe8\x87\xd0\xd2\xeb\x92\xfe\xa6\xd1\xae\xa5l\xbar\x87O\x10>\x94S\xda\x02 ]\xaf\x88\x11\xda\xb0:\x96'QRk\xcf6\x1fIQ*\xea\\v\xd2\x1d\xc5\x99\x96#m\x06s\x00\xf5\x01", b'\x03\xfcb^\x8c%w\x9a9\x9a\xce{\x8f\xbc\x0c\x03p\xee\x886\xae\x9cV\xa4\xc9\xde\xa3\xd0\x06\x80\

One thing I tried here is to mess up the secret key. I provided secret key of our first identity to the first input and secret key of our second identity to the second input. Recall that the first output in the previous transaction is associated with out second identity and second output with our first identity. As such, the secret keys I used were swapped into the wrong order. In this case, the verification should fail. Indeed, when I pushed the transaction to the Bitcoin network, I got the following message:

`sendrawtransaction RPC error: {"code":-26,"message":"mandatory-script-verify-flag-failed (Signature must be zero for failed CHECK(MULTI)SIG operation)"}`

I am not quite sure about the details of this error message but one thing I could tell is that the signature was the problem.

Now, let's try to fix it with the correct secret keys and push it to the network again. The transaction went through successfully and we just wait for its confirmation. It might take at least 10 minutes.

In [143]:
import time; time.sleep(1.0)
# in Bitcoin main net a block will take 10 minutes on average to mine
# Proof of Work difficulty is dynamically adjusted to make it so

Okay, here we have [our transaction](https://www.blockchain.com/btc-testnet/tx/8094f5cd6176dc0cc8f61575c9e6cf13b5f5ebd7a78f1eee70719fc0eb26fbf9) confirmed by the network. The transaction was included in block [2287099](https://www.blockchain.com/btc-testnet/block/2287099) which has a total of 59 transactions. Sweet!!

## Bonus Exercise
In this bonus exercise, we try to steal Andrej's bitcoins from his 3rd wallet (mgh4VjZx5MpkHRis9mDsF2ZcKLdXoP3oQ4) to our first wallet. Currently, [this 3rd wallet](https://www.blockchain.com/btc-testnet/address/mgh4VjZx5MpkHRis9mDsF2ZcKLdXoP3oQ4) has a final balance of 0.00146895 BTC. If done successfully, Andrej's wallet will show "Final Balance" of 0. Let's do it.

First of all, we need to verify something. The final balance of this wallet is 0.00146895 BTC. But the most recent transaction resulted in an
unspent 0.00105923 BTC, not exactly the final balance. So what is this final balance? After doing a little math, I realized that it is the sum of all unspent BTC from previous transacitons. There are total 14 transactions with unspent BTC (most of them are around 3000 sat). If we add all of them up, we will get the final balance of this wallet. This means that we have 14 inputs and one output in this particular transaction that we want to create. 

For instance, the [last transaction](https://www.blockchain.com/btc-testnet/tx/9f49a27b06cdc04c26b08a832bff2c5b0ada9a7bf295b4425d58cfbcdfccf736) resulted in an output of 0.00105932 BTC, which is 105,932 sat. 

Final balance of this address is 146,895 sat. Let's pay 10,000 sat for a fee and send the rest (136,895) to our first address we created.

In [145]:
# ---------------------------------------------------------
# First, we record Andrej's third wallet's public key and private key
secret_key_adj = 29595381593786747354608258168471648998894101022644411057647114205835530364276
public_key_adj = secret_key_adj * G
address_adj = PublicKey.from_point(public_key_adj).address(net='test', compressed=True)

print("Andrej's 3rd Bitcoin identity: ")
print("1. secret key: ", secret_key_adj)
print("2. public key: ", (public_key_adj.x, public_key_adj.y))
print("3. Bitcoin address: ", address_adj)

Andrej's 3rd Bitcoin identity: 
1. secret key:  29595381593786747354608258168471648998894101022644411057647114205835530364276
2. public key:  (10431688308521398859068831048649547920603040245302637088532768399600614938636, 74559974378244821290907538448690356815087741133062157870433812445804889333467)
3. Bitcoin address:  mgh4VjZx5MpkHRis9mDsF2ZcKLdXoP3oQ4


In [171]:
# the inputs of the transaction
# there are total 13 inputs that have unspent BTC

##################################################################################################################
# first input of the transaction
tx_in1 = TxIn(
    prev_tx = bytes.fromhex('9f49a27b06cdc04c26b08a832bff2c5b0ada9a7bf295b4425d58cfbcdfccf736'),
    prev_index = 1,
    script_sig = None, # digital signature to be inserted later
)
# careful that it is the second output in the referened transaction
pkb_hash = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=True)
tx_in1.prev_tx_script_pubkey = Script([118, 169, pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG

# second input of the transaction
tx_in2 = TxIn(
    prev_tx = bytes.fromhex('76ff769a1e4af8b5c76cc245233e091dc98d1173d24a54703222e31e0bde46a1'),
    prev_index = 0,
    script_sig = None, # digital signature to be inserted later
)
# careful that it is the second output in the referened transaction
pkb_hash = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=True)
tx_in2.prev_tx_script_pubkey = Script([118, 169, pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG

# third input of the transaction
tx_in3 = TxIn(
    prev_tx = bytes.fromhex('be5c39844ba8603e3b5a025865938a6ebfe84dcd2bb2cf2b2b2a5d9e530f1cc7'),
    prev_index = 0,
    script_sig = None, # digital signature to be inserted later
)

pkb_hash = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=True)
tx_in3.prev_tx_script_pubkey = Script([118, 169, pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG

# fourth input of the transaction
tx_in4 = TxIn(
    prev_tx = bytes.fromhex('cc06c637501e7dab7ddb3281088dee406970b5361b20949e5cb3106ad8bf04b2'),
    prev_index = 0,
    script_sig = None, # digital signature to be inserted later
)

pkb_hash = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=True)
tx_in4.prev_tx_script_pubkey = Script([118, 169, pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG

# fifth input of the transaction
tx_in5 = TxIn(
    prev_tx = bytes.fromhex('5cb727d9d48815666c4097d36b8d813c6e18e4c4c50f8938752394cb87c6cd52'),
    prev_index = 0,
    script_sig = None, # digital signature to be inserted later
)

pkb_hash = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=True)
tx_in5.prev_tx_script_pubkey = Script([118, 169, pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG

# sixth input of the transaction
tx_in6 = TxIn(
    prev_tx = bytes.fromhex('d46178df22f47403bb9c26c47d669d721e8c922ce86cf79d9a417e7481c5c849'),
    prev_index = 0,
    script_sig = None, # digital signature to be inserted later
)

pkb_hash = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=True)
tx_in6.prev_tx_script_pubkey = Script([118, 169, pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG

# seventh input of the transaction
tx_in7 = TxIn(
    prev_tx = bytes.fromhex('a66ee3c602c99e82bce16c4055f655e528c0a74847699e4c31b660bc104bd35b'),
    prev_index = 0,
    script_sig = None, # digital signature to be inserted later
)

pkb_hash = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=True)
tx_in7.prev_tx_script_pubkey = Script([118, 169, pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG

# eighth input of the transaction
tx_in8 = TxIn(
    prev_tx = bytes.fromhex('44a8c164e9cd09b4f5f65ec41b3cf39fccbf9198dfdeb4c3d3defc0c5f062fb6'),
    prev_index = 0,
    script_sig = None, # digital signature to be inserted later
)

pkb_hash = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=True)
tx_in8.prev_tx_script_pubkey = Script([118, 169, pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG

# ninth input of the transaction
tx_in9 = TxIn(
    prev_tx = bytes.fromhex('242c91511c591feda682a9e32600feb9457ab41a11c69514224b32b29c75ac48'),
    prev_index = 0,
    script_sig = None, # digital signature to be inserted later
)

pkb_hash = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=True)
tx_in9.prev_tx_script_pubkey = Script([118, 169, pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG

# tenth input of the transaction
tx_in10 = TxIn(
    prev_tx = bytes.fromhex('3f32605e933a18f29d09b9579e35f5f97ed0fd18a666f6e79672fee2aab7b2be'),
    prev_index = 0,
    script_sig = None, # digital signature to be inserted later
)

pkb_hash = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=True)
tx_in10.prev_tx_script_pubkey = Script([118, 169, pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG

# eleventh input of the transaction
tx_in11 = TxIn(
    prev_tx = bytes.fromhex('180ff46b7c7c0c4564a181237925caa2d333780874fc85f59fad165eb13b22c0'),
    prev_index = 0,
    script_sig = None, # digital signature to be inserted later
)

pkb_hash = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=True)
tx_in11.prev_tx_script_pubkey = Script([118, 169, pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG

# twelfth input of the transaction
tx_in12 = TxIn(
    prev_tx = bytes.fromhex('0384be704f251fe4fab7b4cf360304586e71101c74caeb934a2f0a94504b7f92'),
    prev_index = 0,
    script_sig = None, # digital signature to be inserted later
)

pkb_hash = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=True)
tx_in12.prev_tx_script_pubkey = Script([118, 169, pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG

# thirteenth input of the transaction
tx_in13 = TxIn(
    prev_tx = bytes.fromhex('599dc4bddecff1d5463f012063df0e1104eaa9350bd872a0bbd4eec283239c8b'),
    prev_index = 0,
    script_sig = None, # digital signature to be inserted later
)

pkb_hash = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=True)
tx_in13.prev_tx_script_pubkey = Script([118, 169, pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG

#####################################################################################################################

# define the output
tx_out = TxOut(
    amount = 100932,
    script_pubkey = None, # locking script, inserted separately below
)

# declare the owner as my first identity above, by inserting the public key hash into the Script 'padding'
out_pkb_hash = PublicKey.from_point(public_key).encode(compressed=True, hash160=True)
out_script = Script([118, 169, out_pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG
tx_out.script_pubkey = out_script

# create the aspiring transaction objext
tx = Tx(
    version = 1,
    # tx_ins = [tx_in1, tx_in2, tx_in3, tx_in4, tx_in5, tx_in6, tx_in7, tx_in8, tx_in9, tx_in10, tx_in11, tx_in12, tx_in13], # total 13 inputs
    tx_ins = [tx_in1],
    tx_outs = [tx_out], # single output
)

# digitally sign the spend of the input, we need to have the Andrej's private key

message1 = tx.encode(sig_index = 1)
random.seed(int.from_bytes(sha256(message1), 'big'))
sig1 = sign(secret_key_adj, message1) # Andrej's third address signs
sig_bytes_and_type1 = sig1.encode() + b'\x01' # DER signature + SIGNHASH_ALL
pubkey_bytes = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=False)
script_sig1 = Script([sig_bytes_and_type1, pubkey_bytes])
tx_in1.script_sig = script_sig1

message2 = tx.encode(sig_index = 0)
random.seed(int.from_bytes(sha256(message2), 'big'))
sig2 = sign(secret_key_adj, message2) # Andrej's third address signs
sig_bytes_and_type2 = sig2.encode() + b'\x01' # DER signature + SIGNHASH_ALL
pubkey_bytes = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=False)
script_sig2 = Script([sig_bytes_and_type2, pubkey_bytes])
tx_in2.script_sig = script_sig2

message3 = tx.encode(sig_index = 0)
random.seed(int.from_bytes(sha256(message3), 'big'))
sig3 = sign(secret_key_adj, message3) # Andrej's third address signs
sig_bytes_and_type3 = sig3.encode() + b'\x01' # DER signature + SIGNHASH_ALL
pubkey_bytes = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=False)
script_sig3 = Script([sig_bytes_and_type3, pubkey_bytes])
tx_in3.script_sig = script_sig3

message4 = tx.encode(sig_index = 0)
random.seed(int.from_bytes(sha256(message4), 'big'))
sig4 = sign(secret_key_adj, message4) # Andrej's third address signs
sig_bytes_and_type4 = sig4.encode() + b'\x01' # DER signature + SIGNHASH_ALL
pubkey_bytes = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=False)
script_sig4 = Script([sig_bytes_and_type4, pubkey_bytes])
tx_in4.script_sig = script_sig4

message5 = tx.encode(sig_index = 0)
random.seed(int.from_bytes(sha256(message5), 'big'))
sig5 = sign(secret_key_adj, message5) # Andrej's third address signs
sig_bytes_and_type5 = sig5.encode() + b'\x01' # DER signature + SIGNHASH_ALL
pubkey_bytes = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=False)
script_sig5 = Script([sig_bytes_and_type5, pubkey_bytes])
tx_in5.script_sig = script_sig5

message6 = tx.encode(sig_index = 0)
random.seed(int.from_bytes(sha256(message6), 'big'))
sig6 = sign(secret_key_adj, message6) # Andrej's third address signs
sig_bytes_and_type6 = sig6.encode() + b'\x01' # DER signature + SIGNHASH_ALL
pubkey_bytes = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=False)
script_sig6 = Script([sig_bytes_and_type6, pubkey_bytes])
tx_in6.script_sig = script_sig6

message7 = tx.encode(sig_index = 0)
random.seed(int.from_bytes(sha256(message7), 'big'))
sig7 = sign(secret_key_adj, message7) # Andrej's third address signs
sig_bytes_and_type7 = sig7.encode() + b'\x01' # DER signature + SIGNHASH_ALL
pubkey_bytes = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=False)
script_sig7 = Script([sig_bytes_and_type7, pubkey_bytes])
tx_in7.script_sig = script_sig7

message8 = tx.encode(sig_index = 0)
random.seed(int.from_bytes(sha256(message8), 'big'))
sig8 = sign(secret_key_adj, message8) # Andrej's third address signs
sig_bytes_and_type8 = sig8.encode() + b'\x01' # DER signature + SIGNHASH_ALL
pubkey_bytes = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=False)
script_sig8 = Script([sig_bytes_and_type8, pubkey_bytes])
tx_in8.script_sig = script_sig8

message9 = tx.encode(sig_index = 0)
random.seed(int.from_bytes(sha256(message9), 'big'))
sig9 = sign(secret_key_adj, message9) # Andrej's third address signs
sig_bytes_and_type9 = sig9.encode() + b'\x01' # DER signature + SIGNHASH_ALL
pubkey_bytes = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=False)
script_sig9 = Script([sig_bytes_and_type9, pubkey_bytes])
tx_in9.script_sig = script_sig9

message10 = tx.encode(sig_index = 0)
random.seed(int.from_bytes(sha256(message10), 'big'))
sig10 = sign(secret_key_adj, message10) # Andrej's third address signs
sig_bytes_and_type10 = sig10.encode() + b'\x01' # DER signature + SIGNHASH_ALL
pubkey_bytes = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=False)
script_sig10 = Script([sig_bytes_and_type10, pubkey_bytes])
tx_in10.script_sig = script_sig10

message11 = tx.encode(sig_index = 0)
random.seed(int.from_bytes(sha256(message11), 'big'))
sig11 = sign(secret_key_adj, message11) # Andrej's third address signs
sig_bytes_and_type11 = sig11.encode() + b'\x01' # DER signature + SIGNHASH_ALL
pubkey_bytes = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=False)
script_sig11 = Script([sig_bytes_and_type11, pubkey_bytes])
tx_in11.script_sig = script_sig11

message12 = tx.encode(sig_index = 0)
random.seed(int.from_bytes(sha256(message12), 'big'))
sig12 = sign(secret_key_adj, message12) # Andrej's third address signs
sig_bytes_and_type12 = sig12.encode() + b'\x01' # DER signature + SIGNHASH_ALL
pubkey_bytes = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=False)
script_sig12 = Script([sig_bytes_and_type12, pubkey_bytes])
tx_in12.script_sig = script_sig12

message13 = tx.encode(sig_index = 0)
random.seed(int.from_bytes(sha256(message13), 'big'))
sig13 = sign(secret_key_adj, message13) # Andrej's third address signs
sig_bytes_and_type13 = sig13.encode() + b'\x01' # DER signature + SIGNHASH_ALL
pubkey_bytes = PublicKey.from_point(public_key_adj).encode(compressed=True, hash160=False)
script_sig13 = Script([sig_bytes_and_type13, pubkey_bytes])
tx_in13.script_sig = script_sig13

# that should be it!
print(tx.id())
print(tx)
print('*' * 125)
print(tx.encode().hex())

56e78607abce704b8c154656690b8c4471986fe97d9236bf8942bdf0a660d6cb
Tx(version=1, tx_ins=[TxIn(prev_tx=b'\x9fI\xa2{\x06\xcd\xc0L&\xb0\x8a\x83+\xff,[\n\xda\x9a{\xf2\x95\xb4B]X\xcf\xbc\xdf\xcc\xf76', prev_index=1, script_sig=Script(cmds=[b"0E\x02!\x00\xb8\x02\xfe\x13\xc9\x96\xd7\\\xc39'\xd3\xbe\xa6\xc3\xf2\xce\xa1\xba\x85B\xa4\xfb\xa0H\x18\xa71\xd8j\xa41\x02 .\xbe\xec\xab\x99\x9c6\xa8C\x87\xabadW\x07\x9d>\xaf\x9a\xf06Dl\xcb\xd6\x8d\x1d+K\xed\xf9\xfd\x01", b'\x03\x17\x10 X\\7#\xa0@)\xe5a\xb8\x9e\xd2\xd5\xed\xf9\x04\xb6\xcaXa\xcd`\x92\xc1L\x88\x04\x98\x0c']), sequence=4294967295)], tx_outs=[TxOut(amount=100932, script_pubkey=Script(cmds=[118, 169, b'\xe0\xa9>\x81q\xc8t\xd2h\x8f\x112\x90\xceB_\xdb\xe4ii', 136, 172]))], locktime=0)
*****************************************************************************************************************************
010000000136f7ccdfbccf585d42b495f27b9ada0a5b2cff2b838ab0264cc0cd067ba2499f010000006b483045022100b802fe13c996d75cc33927d3bea6c3f2cea1ba8542a4f

Okay, here I keep having a problem. I keep getting the error message related to signature. The error message is:

`sendrawtransaction RPC error: {"code":-26,"message":"mandatory-script-verify-flag-failed (Signature must be zero for failed CHECK(MULTI)SIG operation)"}`

I don't understand why it is not working. All of these unspent BTC belong to Andrej's third Bitcoin wallet and I have all the information, including secret key, 
public key, and address. For all messages, I am supposed to use the same secret key. But the error message is basically indicating that the signature is not valid.

Another question I have after implementing this is that all those 13 transactions use the same secret key. In that case, can we simplify this process and just sign once?

---
Okay, I think I will stop here now. This has been an extremely valuable exploration. I don't know how Bitcoin actually works prior to this implementation. Now I have much much better grasp about the Bitcoin protocol. 